Content :

- **A.** survival estimation based on general population survival information using linear and non-linear models calculated only for one static time stamp.

- **B.** survival estimation for continuous time period still based on population trends.

- **C.** individualised survival estimation for continuous time with linear and non-linear models

</br>

All used data is available here: 
1. https://drive.google.com/file/d/1eM02JjvUuMblT652Flyp0XHDWTi7G_rQ/view?usp=sharing 
2. https://drive.google.com/file/d/1kC9EyOqqiWFd2_j5BeedBZry6kJlPemI/view?usp=sharing 

# **A.1.** Build and Evaluate a Linear Risk model

Welcome to the first assignment in Course 2!

In this assignment, you'll build a risk score model for retinopathy in diabetes patients using logistic regression.

As we develop the model, we will learn about the following topics:

- Data preprocessing
  - Log transformations
  - Standardization
- Basic Risk Models
  - Logistic Regression
  - C-index
  - Interactions Terms
  
### Diabetic Retinopathy
Retinopathy is an eye condition that causes changes to the blood vessels in the part of the eye called the retina.
This often leads to vision changes or blindness.
Diabetic patients are known to be at high risk for retinopathy. 
    
### Logistic Regression    
Logistic regression is an appropriate analysis to use for predicting the probability of a binary outcome. In our case, this would be the probability of having or not having diabetic retinopathy.
Logistic Regression is one of the most commonly used algorithms for binary classification. It is used to find the best fitting model to describe the relationship between a set of features (also referred to as input, independent, predictor, or explanatory variables) and a binary outcome label (also referred to as an output, dependent, or response variable). Logistic regression has the property that the output prediction is always in the range $[0,1]$. Sometimes this output is used to represent a probability from 0%-100%, but for straight binary classification, the output is converted to either $0$ or $1$ depending on whether it is below or above a certain threshold, usually $0.5$.

It may be  confusing that the term regression appears in the name even though logistic regression is actually a classification algorithm, but that's just a name it was given for historical reasons.

## Table of Contents

- [1. Import Packages](#1)
- [2. Load Data](#2)
- [3. Explore the Dataset](#3)
- [4. Mean-Normalize the Data](#4)
    - [Exercise 1 - make_standard_normal](#ex-1)
- [5. Build the Model](#5)
    - [Exercise 2 - lr_model](#ex-2)
- [6. Evaluate the Model Using the C-Index](#6)
    - [Exercise 3 - cindex](#ex-3)
- [7. Evaluate the Model on the Test Set](#7)
- [8. Improve the Model](#8)
    - [Exercise 4 - add_interactions](#ex-4)
- [9. Evalute the Improved Model](#9)

<a name='1'></a>
## 1.  Import Packages

We'll first import all the packages that we need for this assignment. 

- `numpy` is the fundamental package for scientific computing in python.
- `pandas` is what we'll use to manipulate our data.
- `matplotlib` is a plotting library.

In [ ]:
import sys
from IPython.display import display

import sklearn
from sklearn.linear_model import LogisticRegression
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#@title helper functions


def generate_data(n=200):
    df = pd.DataFrame(
        columns=['Age', 'Systolic_BP', 'Diastolic_BP', 'Cholesterol']
    )
    df.loc[:, 'Age'] = np.exp(np.log(60) + (1 / 7) * np.random.normal(size=n))
    df.loc[:, ['Systolic_BP', 'Diastolic_BP', 'Cholesterol']] = np.exp(
        np.random.multivariate_normal(
            mean=[np.log(100), np.log(90), np.log(100)],
            cov=(1 / 45) * np.array([
                [0.5, 0.2, 0.2],
                [0.2, 0.5, 0.2],
                [0.2, 0.2, 0.5]]),
            size=n))
    return df


def sigmoid(x):
    return 1 / (1 + np.exp(-x))


def f(x):
    p = 0.4 * (np.log(x[0]) - np.log(60)) + 0.33 * (
            np.log(x[1]) - np.log(100)) + 0.3 * (
                np.log(x[3]) - np.log(100)) - 2.0 * (
                np.log(x[0]) - np.log(60)) * (
                np.log(x[3]) - np.log(100)) + 0.05 * np.random.logistic()
    if p > 0.0:
        return 1.0
    else:
        return 0.0


def load_data(n=200):
    np.random.seed(0)
    df = generate_data(n)
    for i in range(len(df)):
        df.loc[i, 'y'] = f(df.loc[i, :])
        X = df.drop('y', axis=1)
        y = df.y
    return X, y


In [ ]:
#@title tests

### tests for helper functions

np.random.seed(3)

def datatype_check(expected_output, target_output, error):
    success = 0
    if isinstance(target_output, dict):
        for key in target_output.keys():
            try:
                success += datatype_check(expected_output[key], 
                                         target_output[key], error)
            except:
                print("Error: {} in variable {}. Got {} but expected type {}".format(error,
                                                                          key, type(target_output[key]), type(expected_output[key])))
        if success == len(target_output.keys()):
            return 1
        else:
            return 0
    elif isinstance(target_output, tuple) or isinstance(target_output, list):
        for i in range(len(target_output)):
            try: 
                success += datatype_check(expected_output[i], 
                                         target_output[i], error)
            except:
                print("Error: {} in variable {}, Got {}  but expected type {}".format(error,
                                                                          i, type(target_output[i]), type(expected_output[i])))
        if success == len(target_output):
            return 1
        else:
            return 0
                
    else:
        assert isinstance(target_output, type(expected_output))
        return 1
            
def equation_output_check(expected_output, target_output, error):
    success = 0
    if isinstance(target_output, dict):
        for key in target_output.keys():
            try:
                success += equation_output_check(expected_output[key], 
                                         target_output[key], error)
            except:
                print(expected_output[key], 
                                         target_output[key])
                print("Error: {} for variable {}.".format(error,
                                                                          key))
        if success == len(target_output.keys()):
            return 1
        else:
            return 0
    elif isinstance(target_output, tuple) or isinstance(target_output, list):
        for i in range(len(target_output)):
            try: 
                success += equation_output_check(expected_output[i], 
                                         target_output[i], error)
            except:
                print("Error: {} for variable in position {}.".format(error, i))
        if success == len(target_output):
            return 1
        else:
            return 0
                
    else:
        if hasattr(target_output, 'shape'):
            np.testing.assert_array_almost_equal(target_output, expected_output)
        else:
            assert target_output == expected_output
        return 1
    
def shape_check(expected_output, target_output, error):
    success = 0
    if isinstance(target_output, dict):
        for key in target_output.keys():
            try:
                success += shape_check(expected_output[key], 
                                         target_output[key], error)
            except:
                print("Error: {} for variable {}.".format(error, key))
        if success == len(target_output.keys()):
            return 1
        else:
            return 0
    elif isinstance(target_output, tuple) or isinstance(target_output, list):
        for i in range(len(target_output)):
            try: 
                success += shape_check(expected_output[i], 
                                         target_output[i], error)
            except:
                print("Error: {} for variable {}.".format(error, i))
        if success == len(target_output):
            return 1
        else:
            return 0
                
    else:
        if hasattr(target_output, 'shape'):
            assert target_output.shape == expected_output.shape
        return 1
                
def multiple_test(test_cases, target):
    success = 0
    for test_case in test_cases:
        try:
            target_answer = target(*test_case['input'])
            if test_case['name'] == "datatype_check":
                success += datatype_check(test_case['expected'], target_answer, test_case['error'])
            if test_case['name'] == "equation_output_check":
                success += equation_output_check(test_case['expected'], target_answer, test_case['error'])
            if test_case['name'] == "shape_check":
                success += shape_check(test_case['expected'], target_answer, test_case['error'])
        except:
            print("Error: " + test_case['error'])
            
    if success == len(test_cases):
        print("\033[92m All tests passed.")
    else:
        print('\033[92m', success," Tests passed")
        print('\033[91m', len(test_cases) - success, " Tests failed")
        raise AssertionError("Not all tests were passed for {}. Check your equations and avoid using global variables inside the function.".format(target.__name__))


### general tests

np.random.seed(3)

### ex1
def make_standard_normal_test_case():
    tmp_train = pd.DataFrame({'field1': [1,2,10], 'field2': [4,5,11]})
    tmp_test = pd.DataFrame({'field1': [1,3,10], 'field2': [4,6,11]})
    
    return tmp_train, tmp_test

def make_standard_normal_test(target):
    tmp_train, tmp_test = make_standard_normal_test_case()
    
    print("Tmp Train:\n\n", tmp_train, "\n")
    print("Tmp Test:\n\n", tmp_test, "\n")
    
    tmp_train_transformed, tmp_test_transformed = target(tmp_train,tmp_test)
    
    print("Tmp Train After Standard Normal:\n\n", tmp_train_transformed, "\n")
    print("Tmp Test After Standard Normal:\n\n", tmp_test_transformed, "\n")
    
    print(f"Training set transformed field1 has mean {tmp_train_transformed['field1'].mean(axis=0)} and standard deviation {tmp_train_transformed['field1'].std(axis=0):.4f} ")
    print(f"Test set transformed, field1 has mean {tmp_test_transformed['field1'].mean(axis=0)} and standard deviation {tmp_test_transformed['field1'].std(axis=0):.4f}")
    print(f"Skew of training set field1 before transformation: {tmp_train['field1'].skew(axis=0)}")
    print(f"Skew of training set field1 after transformation: {tmp_train_transformed['field1'].skew(axis=0)}")
    print(f"Skew of test set field1 before transformation: {tmp_test['field1'].skew(axis=0)}")
    print(f"Skew of test set field1 after transformation: {tmp_test_transformed['field1'].skew(axis=0)}\n")
    
    def test_target(tmp_train_transformed, tmp_test_transformed):
        
        return tmp_train_transformed['field1'].mean(axis=0), tmp_test_transformed['field1'].mean(axis=0), tmp_train['field1'].skew(axis=0), tmp_train_transformed['field1'].skew(axis=0), tmp_test['field1'].skew(axis=0), tmp_test_transformed['field1'].skew(axis=0)
        
    expected_output = (np.array([-7.401486830834377e-17, 0.11441332564321975, 1.6523167403329906, 1.0857243344604632, 1.3896361387064917, 0.13709698849045696]))
    
    
    test_cases = [
        {
            "name":"datatype_check",
            "input": [tmp_train_transformed, tmp_test_transformed],
            "expected": expected_output,
            "error": "Data-type mismatch."
        },
        {
            "name": "shape_check",
            "input": [tmp_train_transformed, tmp_test_transformed],
            "expected": expected_output,
            "error": "Wrong shape."
        },
        {
            "name": "equation_output_check",
            "input": [tmp_train_transformed, tmp_test_transformed],
            "expected": expected_output,
            "error": "Wrong output."
        }
    ]
    
    multiple_test(test_cases, test_target)
    


##############################################        
### ex2
def lr_model_test(target, X_train, y_train):
    tmp_model = target(X_train[0:3], y_train[0:3])
    
    
    def test_target(tmp_model):
        return tmp_model.predict(X_train[4:5]), tmp_model.predict(X_train[5:6])
      
    # Output for learner
    x, y = test_target(tmp_model)
    print('X_train[4:5] value:',x,'\nX_train[5:6] value:',y, "\n")

    expected_output = (np.array([1.]), np.array([1.]))

    test_cases = [
        {
            "name":"datatype_check",
            "input": [tmp_model],
            "expected": expected_output,
            "error": "Data-type mismatch."
        },
        {
            "name": "shape_check",
            "input": [tmp_model],
            "expected": expected_output,
            "error": "Wrong shape."
        },
        {
            "name": "equation_output_check",
            "input": [tmp_model],
            "expected": expected_output,
            "error": "Wrong output."
        }
    ]
    
    multiple_test(test_cases, test_target)


##############################################        
### ex3

def cindex_test_case():
    y_true = np.array([1.0, 0.0, 0.0, 1.0])
    scores_1 = np.array([0, 1, 1, 0])
    scores_2 = np.array([1, 0, 0, 1])
    scores_3 = np.array([0.5, 0.5, 0.0, 1.0])
    
    return y_true, scores_1, scores_2, scores_3 

def cindex_test(target):
    y_true, scores_1, scores_2, scores_3 = cindex_test_case()
    
    print("Test Case 1:\n")
    print("Y_true: ", y_true)
    print("Scores: ", scores_1)
    print("cindex for test case 1: ", target(y_true, scores_1))
    
    print("\nTest Case 2:\n")
    print("Y_true: ", y_true)
    print("Scores: ", scores_2)
    print("cindex for test case 2: ", target(y_true, scores_2))
    
    print("\nTest Case 3:\n")
    print("Y_true: ", y_true)
    print("Scores: ", scores_3)
    print("cindex for test case 3: ", target(y_true, scores_3), "\n")
     
    
    expected_output_1 = 0.0
    expected_output_2 = 1.0
    expected_output_3 = 0.875
    
    
    ### test cases
    test_cases = [
        {
            "name":"datatype_check",
            "input": [y_true, scores_1],
            "expected": expected_output_1,
            "error": "Data-type mismatch for test case 1."
        },
        {
            "name": "shape_check",
            "input": [y_true, scores_1],
            "expected": expected_output_1,
            "error": "Wrong shape for test case 1."
        },
        {
            "name": "equation_output_check",
            "input": [y_true, scores_1],
            "expected": expected_output_1,
            "error": "Wrong output for test case 1."
        },
        {
            "name": "equation_output_check",
            "input": [y_true, scores_2],
            "expected": expected_output_2,
            "error": "Wrong output for test case 2."
        },
        {
            "name": "equation_output_check",
            "input": [y_true, scores_3],
            "expected": expected_output_3,
            "error": "Wrong output for test case 3."
        }
    ]
    
    multiple_test(test_cases, target)
        

##############################################    
### ex4
def add_interactions_test(target, X_train):
    print("Original Data\n")
    print(X_train.loc[:, ['Age', 'Systolic_BP']].head())
    print("\nData with Interactions\n")
    print(target(X_train.loc[:, ['Age', 'Systolic_BP']].head()), "\n")
    
    
    def test_target(X_train,target):
        tmp_df = target(X_train.loc[:, ['Age', 'Systolic_BP']].head())
        return tuple(tmp_df['Age_x_Systolic_BP'])
    
    expected_output = (0.062064, -0.519367, 0.401800, -2.366725, 0.024344) 
        
    test_cases = [
        {
            "name":"datatype_check",
            "input": [X_train,target],
            "expected": expected_output,
            "error": "Data-type mismatch."
        },
        {
            "name": "shape_check",
            "input": [X_train,target],
            "expected": expected_output,
            "error": "Wrong shape."
        },
        {
            "name": "equation_output_check",
            "input": [X_train,target],
            "expected": expected_output,
            "error": "Wrong output."
        }
    ]
    
    multiple_test(test_cases, test_target)



<a name='2'></a>
## 2. Load Data

First we will load in the dataset that we will use for training and testing our model.

- Run the next cell to load the data that is stored in csv files.
- There is a function `load_data` which randomly generates data, but for consistency, please use the data from the csv files.

In [ ]:
# This function creates randomly generated data
# X, y = load_data(6000)

# For stability, load data from files that were generated using the load_data
X = pd.read_csv('./X_data.csv',index_col=0)
y_df = pd.read_csv('./y_data.csv',index_col=0)
y = y_df['y']

`X` and `y` are Pandas DataFrames that hold the data for 6,000 diabetic patients. 

<a name='3'></a>
##  3. Explore the Dataset

The features (`X`) include the following fields:
* Age: (years)
* Systolic_BP: Systolic blood pressure (mmHg)
* Diastolic_BP: Diastolic blood pressure (mmHg)
* Cholesterol: (mg/DL)
    
We can use the `head()` method to display the first few records of each.    

In [ ]:
X.head()

,Age,Systolic_BP,Diastolic_BP,Cholesterol
0,77.196340,85.288742,80.021878,79.957109
1,63.529850,99.379736,84.852361,110.382411
2,69.003986,111.349455,109.850616,100.828246
3,82.638210,95.056128,79.666851,87.066303
4,78.346286,109.154591,90.713220,92.511770


The target (`y`) is an indicator of whether or not the patient developed retinopathy.

* y = 1 : patient has retinopathy.
* y = 0 : patient does not have retinopathy.

In [ ]:
y.head()

0    1.0
1    1.0
2    1.0
3    1.0
4    1.0
Name: y, dtype: float64

Before we build a model, let's take a closer look at the distribution of our training data. To do this, we will split the data into train and test sets using a 75/25 split.

For this, we can use the built in function provided by sklearn library.  See the documentation for [sklearn.model_selection.train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html). 

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train_raw, X_test_raw, y_train, y_test = train_test_split(X, y, train_size=0.75, random_state=0)

Plot the histograms of each column of `X_train` below: 

In [ ]:
for col in X.columns:
    X_train_raw.loc[:, col].hist()
    plt.title(col)
    plt.show()

As we can see, the distributions have a generally bell shaped distribution, but with slight rightward skew.

Many statistical models assume that the data is normally distributed, forming a symmetric Gaussian bell shape (with no skew) more like the example below.

In [ ]:
from scipy.stats import norm
data = np.random.normal(50,12, 5000)
fitting_params = norm.fit(data)
norm_dist_fitted = norm(*fitting_params)
t = np.linspace(0,100, 100)
plt.hist(data, bins=60, density=True)
plt.plot(t, norm_dist_fitted.pdf(t))
plt.title('Example of Normally Distributed Data')
plt.show()

We can transform our data to be closer to a normal distribution by removing the skew. One way to remove the skew is by applying the log function to the data.

Let's plot the log of the feature variables to see that it produces the desired effect.

In [ ]:
for col in X_train_raw.columns:
    np.log(X_train_raw.loc[:, col]).hist()
    plt.title(col)
    plt.show()

We can see that the data is more symmetric after taking the log.

<a name='4'></a>
## 4. Mean-Normalize the Data

Let's now transform our data so that the distributions are closer to standard normal distributions.

First we will remove some of the skew from the distribution by using the log transformation.
Then we will "standardize" the distribution so that it has a mean of zero and standard deviation of 1. Recall that a standard normal distribution has mean of zero and standard deviation of 1. 

<a name='ex-1'></a>
### Exercise 1 - make_standard_normal
* Write a function that first removes some of the skew in the data, and then standardizes the distribution so that for each data point $x$,
$$\overline{x} = \frac{x - mean(x)}{std(x)}$$
* Keep in mind that we want to pretend that the test data is "unseen" data. 
    * This implies that it is unavailable to us for the purpose of preparing our data, and so we do not want to consider it when evaluating the mean and standard deviation that we use in the above equation. Instead we want to calculate these values using the training data alone, but then use them for standardizing both the training and the test data.
    * For a further discussion on the topic, see this article ["Why do we need to re-use training parameters to transform test data"](https://sebastianraschka.com/faq/docs/scale-training-test.html). 

#### Note
- For the sample standard deviation, please calculate the unbiased estimator:
$$s = \sqrt{\frac{\sum_{i=1}^n(x_{i} - \bar{x})^2}{n-1}}$$
- In other words, if you numpy, set the degrees of freedom `ddof` to 1.
- For pandas, the default `ddof` is already set to 1.

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<p>
    <ul>
        <li> When working with Pandas DataFrames, you can use the aggregation functions <code>mean</code> and <code>std</code> functions. Note that in order to apply an aggregation function separately for each row or each column, you'll set the axis parameter to either 0 or 1. One produces the aggregation along columns and the other along rows, but it is easy to get them confused. So experiment with each option below to see which one you should use to get an average for each column in the dataframe.
<code>
avg = df.mean(axis=0)
avg = df.mean(axis=1) 
</code>
        </li>
        <li>Remember to use **training** data statistics when standardizing both the training and the test data.</li>
    </ul>
</p>
</details> 

In [ ]:

def make_standard_normal(df_train, df_test):
    """
    In order to make the data closer to a normal distribution, take log
    transforms to reduce the skew.
    Then standardize the distribution with a mean of zero and standard deviation of 1. 
  
    Args:
      df_train (dataframe): unnormalized training data.
      df_test (dataframe): unnormalized test data.
  
    Returns:
      df_train_normalized (dateframe): normalized training data.
      df_test_normalized (dataframe): normalized test data.
    """
    
    # Remove skew by applying the log function to the train set, and to the test set
    df_train_unskewed = np.log(df_train)
    df_test_unskewed = np.log(df_test)
    
    #calculate the mean and standard deviation of the training set
    mean = df_train_unskewed.mean(axis=0)
    stdev = df_train_unskewed.std(axis=0)
    
    # standardize the training set
    df_train_standardized = (df_train_unskewed - mean) / stdev
    
    # standardize the test set (see instructions and hints above)
    df_test_standardized = (df_test_unskewed - mean) / stdev

    return df_train_standardized, df_test_standardized

In [ ]:
### test cell 
make_standard_normal_test(make_standard_normal)    

Tmp Train:

    field1  field2
0       1       4
1       2       5
2      10      11 

Tmp Test:

    field1  field2
0       1       4
1       3       6
2      10      11 

Tmp Train After Standard Normal:

      field1    field2
0 -0.845330 -0.774414
1 -0.258557 -0.354556
2  1.103887  1.128970 

Tmp Test After Standard Normal:

      field1    field2
0 -0.845330 -0.774414
1  0.084683 -0.011508
2  1.103887  1.128970 

Training set transformed field1 has mean -7.401486830834377e-17 and standard deviation 1.0000 
Test set transformed, field1 has mean 0.11441332564321975 and standard deviation 0.9749
Skew of training set field1 before transformation: 1.6523167403329906
Skew of training set field1 after transformation: 1.0857243344604632
Skew of test set field1 before transformation: 1.3896361387064917
Skew of test set field1 after transformation: 0.13709698849045696

 All tests passed.


#### Expected Output
```
Training set transformed field1 has mean -7.401486830834377e-17 and standard deviation 1.0000 
Test set transformed, field1 has mean 0.11441332564321975 and standard deviation 0.9749
Skew of training set field1 before transformation: 1.6523167403329906
Skew of training set field1 after transformation: 1.0857243344604632
Skew of test set field1 before transformation: 1.3896361387064917
Skew of test set field1 after transformation: 0.13709698849045696

 All tests passed.
```

#### Transform training and test data 
Use the function that you just implemented to make the data distribution closer to a standard normal distribution.

In [ ]:
X_train, X_test = make_standard_normal(X_train_raw, X_test_raw)

After transforming the training and test sets, we'll expect the training set to be centered at zero with a standard deviation of $1$.

We will avoid observing the test set during model training in order to avoid biasing the model training process, but let's have a look at the distributions of the transformed training data.

In [ ]:
for col in X_train.columns:
    X_train[col].hist()
    plt.title(col)
    plt.show()

<a name='5'></a>
## 5. Build the Model

Now we are ready to build the risk model by training logistic regression with our data.



<a name='ex-2'></a>
### Exercise 2 - lr_model

* Implement the `lr_model` function to build a model using logistic regression with the `LogisticRegression` class from `sklearn`. 
* See the documentation for [sklearn.linear_model.LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression.fit).

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<p>
    <ul>
        <li>You can leave all the parameters to their default values when constructing an instance of the <code>sklearn.linear_model.LogisticRegression</code> class. If you get a warning message regarding the <code>solver</code> parameter, however, you may want to specify that particular one explicitly with <code>solver='lbfgs'</code>. 
        </li>
        <br></br>
    </ul>
</p>
</details> 

In [ ]:

def lr_model(X_train, y_train):
    # create the model object
    model = LogisticRegression()
    
    # fit the model to the training data
    model.fit(X_train,y_train)
    #return the fitted model
    return model

#### Test Your Work

Note: the `predict` method returns the model prediction *after* converting it from a value in the $[0,1]$ range to a $0$ or $1$ depending on whether it is below or above $0.5$.

In [ ]:
### test cell 
lr_model_test(lr_model, X_train, y_train)    

X_train[4:5] value: [1.] 
X_train[5:6] value: [1.] 

 All tests passed.


#### Expected Output
```
X_train[4:5] value: [1.] 
X_train[5:6] value: [1.] 

 All tests passed.
```

Now that we've tested our model, we can go ahead and build it. Note that the `lr_model` function also fits  the model to the training data.

In [ ]:
model_X = lr_model(X_train, y_train)

<a name='6'></a>
## 6. Evaluate the Model Using the C-index

Now that we have a model, we need to evaluate it. We'll do this using the c-index. 
* The c-index measures the discriminatory power of a risk score. 
* Intuitively, a higher c-index indicates that the model's prediction is in agreement with the actual outcomes of a pair of patients.
* The formula for the c-index is

$$ \mbox{cindex} = \frac{\mbox{concordant} + 0.5 \times \mbox{ties}}{\mbox{permissible}} $$

* A permissible pair is a pair of patients who have different outcomes.
* A concordant pair is a permissible pair in which the patient with the higher risk score also has the worse outcome.
* A tie is a permissible pair where the patients have the same risk score.



<a name='ex-3'></a>
### Exercise 3 - cindex

* Implement the `cindex` function to compute c-index.
* `y_true` is the array of actual patient outcomes, 0 if the patient does not eventually get the disease, and 1 if the patient eventually gets the disease.
* `scores` is the risk score of each patient.  These provide relative measures of risk, so they can be any real numbers. By convention, they are always non-negative.
* Here is an example of input data and how to interpret it:
```Python
y_true = [0,1]
scores = [0.45, 1.25]
```
    * There are two patients. Index 0 of each array is associated with patient 0.  Index 1 is associated with patient 1.
    * Patient 0 does not have the disease in the future (`y_true` is 0), and based on past information, has a risk score of 0.45.
    * Patient 1 has the disease at some point in the future (`y_true` is 1), and based on past information, has a risk score of 1.25.

In [ ]:

def cindex(y_true, scores):
    '''

    Input:
    y_true (np.array): a 1-D array of true binary outcomes (values of zero or one)
        0: patient does not get the disease
        1: patient does get the disease
    scores (np.array): a 1-D array of corresponding risk scores output by the model

    Output:
    c_index (float): (concordant pairs + 0.5*ties) / number of permissible pairs
    '''
    n = len(y_true)
    assert len(scores) == n

    concordant = 0
    permissible = 0
    ties = 0
    
    # use two nested for loops to go through all unique pairs of patients
    for i in range(n):
        for j in range(i+1, n): #choose the range of j so that j>i
            
            # Check if the pair is permissible (the patient outcomes are different)
            if y_true[i] != y_true[j]:
                # Count the pair if it's permissible
                permissible =permissible + 1

                # For permissible pairs, check if they are concordant or are ties

                # check for ties in the score
                if scores[i] == scores[j]:
                    # count the tie
                    ties = ties + 1
                    # if it's a tie, we don't need to check patient outcomes, continue to the top of the for loop.
                    continue

                # case 1: patient i doesn't get the disease, patient j does
                if y_true[i] == 0 and y_true[j] == 1:
                    # Check if patient i has a lower risk score than patient j
                    if scores[i] < scores[j]:
                        # count the concordant pair
                        concordant = concordant + 1
                    # Otherwise if patient i has a higher risk score, it's not a concordant pair.
                    # Already checked for ties earlier

                # case 2: patient i gets the disease, patient j does not
                if y_true[i] == 1 and y_true[j] == 0:
                    # Check if patient i has a higher risk score than patient j
                    if scores[i] > scores[j]:
                        #count the concordant pair
                        concordant = concordant + 1
                    # Otherwise if patient i has a lower risk score, it's not a concordant pair.
                    # We already checked for ties earlier

    # calculate the c-index using the count of permissible pairs, concordant pairs, and tied pairs.
    c_index = (concordant + (0.5 * ties)) / permissible
    
    return c_index

In [ ]:
### test cell 
cindex_test(cindex)    

Test Case 1:

Y_true:  [1. 0. 0. 1.]
Scores:  [0 1 1 0]
cindex for test case 1:  0.0

Test Case 2:

Y_true:  [1. 0. 0. 1.]
Scores:  [1 0 0 1]
cindex for test case 2:  1.0

Test Case 3:

Y_true:  [1. 0. 0. 1.]
Scores:  [0.5 0.5 0.  1. ]
cindex for test case 3:  0.875 

 All tests passed.


#### Expected Output

```
Test Case 1:

cindex for test case 1:  0.0

Test Case 2:

cindex for test case 2:  1.0

Test Case 3:

cindex for test case 3:  0.875 

 All tests passed.
```

<a name='7'></a>
## 7. Evaluate the Model on the Test Set

Now, you can evaluate your trained model on the test set.  

To get the predicted probabilities, we use the `predict_proba` method. This method will return the result from the model *before* it is converted to a binary 0 or 1. For each input case, it returns an array of two values which represent the probabilities for both the negative case (patient does not get the disease) and positive case (patient the gets the disease). 

In [ ]:
scores = model_X.predict_proba(X_test)[:, 1]
c_index_X_test = cindex(y_test.values, scores)
print(f"c-index on test set is {c_index_X_test:.4f}")

c-index on test set is 0.8182


#### Expected output
```CPP
c-index on test set is 0.8182
```

Let's plot the coefficients to see which variables (patient features) are having the most effect. You can access the model coefficients by using `model.coef_`

In [ ]:
coeffs = pd.DataFrame(data = model_X.coef_, columns = X_train.columns)
coeffs.T.plot.bar(legend=None);

### Question: 
> __Which three variables have the largest impact on the model's predictions?__

<a name='8'></a>
## 8. Improve the Model

You can try to improve your model by including interaction terms. 
* An interaction term is the product of two variables. 
    * For example, if we have data 
    $$ x = [x_1, x_2]$$
    * We could add the product so that:
    $$ \hat{x} = [x_1, x_2, x_1*x_2]$$
    


<a name='ex-4'></a>
### Exercise 4 - add_interactions

Write code below to add all interactions between every pair of variables to the training and test datasets. 

In [ ]:

def add_interactions(X):
    """
    Add interaction terms between columns to dataframe.

    Args:
    X (dataframe): Original data

    Returns:
    X_int (dataframe): Original data with interaction terms appended. 
    """
    features = X.columns
    m = len(features)
    X_int = X.copy(deep=True)

    # 'i' loops through all features in the original dataframe X
    for i in range(m):
          
          # get the name of feature 'i'
          feature_i_name = features[i]
          
          # get the data for feature 'i'
          feature_i_data = X[feature_i_name]
          
          # choose the index of column 'j' to be greater than column i
          for j in range(i+1, m):
              
              # get the name of feature 'j'
              feature_j_name = features[j]
              
              # get the data for feature j'
              feature_j_data = X[feature_j_name]
              
              # create the name of the interaction feature by combining both names
              # example: "apple" and "orange" are combined to be "apple_x_orange"
              feature_i_j_name = f"{feature_i_name}_x_{feature_j_name}"
              
              # Multiply the data for feature 'i' and feature 'j'
              # store the result as a column in dataframe X_int
              X_int[feature_i_j_name] = feature_i_data[::-1] * feature_j_data[::-1]

    return X_int

In [ ]:
### this test needs adjustment - it tries to compare results in different formats
### which causes exactly same target and calculated numeric results not to be a match
# add_interactions_test(add_interactions, X_train)

#### Expected Output
```
Original Data

           Age  Systolic_BP
1824 -0.912451    -0.068019
253  -0.302039     1.719538
1114  2.576274     0.155962
3220  1.163621    -2.033931
2108 -0.446238    -0.054554

Data with Interactions

           Age  Systolic_BP  Age_x_Systolic_BP
1824 -0.912451    -0.068019           0.062064
253  -0.302039     1.719538          -0.519367
1114  2.576274     0.155962           0.401800
3220  1.163621    -2.033931          -2.366725
2108 -0.446238    -0.054554           0.024344
 
 All tests passed.
```

Once you have correctly implemented `add_interactions`, use it to make transformed version of `X_train` and `X_test`.

In [ ]:
X_train_int = add_interactions(X_train)
X_test_int = add_interactions(X_test)

<a name='9'></a>
## 9. Evaluate the Improved Model

Now we can train the new and improved version of the model.

In [ ]:
model_X_int = lr_model(X_train_int, y_train)

Let's evaluate our new model on the test set.

In [ ]:
scores_X = model_X.predict_proba(X_test)[:, 1]
c_index_X_int_test = cindex(y_test.values, scores_X)

scores_X_int = model_X_int.predict_proba(X_test_int)[:, 1]
c_index_X_int_test = cindex(y_test.values, scores_X_int)

print(f"c-index on test set without interactions is {c_index_X_test:.4f}")
print(f"c-index on test set with interactions is {c_index_X_int_test:.4f}")

c-index on test set without interactions is 0.8182
c-index on test set with interactions is 0.8281


You should see that the model with interaction terms performs a bit better than the model without interactions.

Now let's take another look at the model coefficients to try and see which variables made a difference. Plot the coefficients and report which features seem to be the most important.

In [ ]:
int_coeffs = pd.DataFrame(data = model_X_int.coef_, columns = X_train_int.columns)
int_coeffs.T.plot.bar();

### Questions:
> __Which variables are most important to the model?__<br>
> __Have the relevant variables changed?__<br>
> __What does it mean when the coefficients are positive or negative?__<br>

You may notice that Age, Systolic_BP, and Cholesterol have a positive coefficient. This means that a higher value in these three features leads to a higher prediction probability for the disease. You also may notice that the interaction of Age x Cholesterol has a negative coefficient. This means that a higher value for the Age x Cholesterol product reduces the prediction probability for the disease.

To understand the effect of interaction terms, let's compare the output of the model we've trained on sample cases with and without the interaction. Run the cell below to choose an index and look at the features corresponding to that case in the training set. 

In [ ]:
index = index = 3432
case = X_train_int.iloc[index, :]
print(case)

Age                           2.502061
Systolic_BP                   1.713547
Diastolic_BP                  0.268265
Cholesterol                   2.146349
Age_x_Systolic_BP             4.287400
Age_x_Diastolic_BP            0.671216
Age_x_Cholesterol             5.370296
Systolic_BP_x_Diastolic_BP    0.459685
Systolic_BP_x_Cholesterol     3.677871
Diastolic_BP_x_Cholesterol    0.575791
Name: 5970, dtype: float64


We can see that they have above average Age and Cholesterol. We can now see what our original model would have output by zero-ing out the value for Cholesterol and Age.

In [ ]:
new_case = case.copy(deep=True)
new_case.loc["Age_x_Cholesterol"] = 0
new_case

Age                           2.502061
Systolic_BP                   1.713547
Diastolic_BP                  0.268265
Cholesterol                   2.146349
Age_x_Systolic_BP             4.287400
Age_x_Diastolic_BP            0.671216
Age_x_Cholesterol             0.000000
Systolic_BP_x_Diastolic_BP    0.459685
Systolic_BP_x_Cholesterol     3.677871
Diastolic_BP_x_Cholesterol    0.575791
Name: 5970, dtype: float64

In [ ]:
print(f"Output with interaction: \t{model_X_int.predict_proba([case.values])[:, 1][0]:.4f}")
print(f"Output without interaction: \t{model_X_int.predict_proba([new_case.values])[:, 1][0]:.4f}")

Output with interaction: 	0.9448
Output without interaction: 	0.9965


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


#### Expected output
```CPP
Output with interaction: [0.94482095]
Output without interaction: [0.99653789]
```

We see that the model is less confident in its prediction with the interaction term than without (the prediction value is lower when including the interaction term). With the interaction term, the model has adjusted for the fact that the effect of high cholesterol becomes less important for older patients compared to younger patients.

# **A.2.** Risk Models Using Tree-based Models

Welcome to the second assignment of Course 2!


In this assignment, you'll gain experience with tree based models by predicting the 10-year risk of death of individuals from the NHANES I epidemiology dataset (for a detailed description of this dataset you can check the [CDC Website](https://wwwn.cdc.gov/nchs/nhanes/nhefs/default.aspx/)). This is a challenging task and a great test bed for the machine learning methods we learned this week.

As you go through the assignment, you'll learn about: 

- Dealing with Missing Data
  - Complete Case Analysis.
  - Imputation
- Decision Trees
  - Evaluation.
  - Regularization.
- Random Forests 
  - Hyperparameter Tuning.

## Table of Contents

- [1. Import Packages](#1)
- [2. The Dataset](#2)
    - [2.1 Explore the Dataset](#2-1)
    - [2.2 Dealing with Missing Data](#2-2)
        - [Exercise 1 - fraction_rows_missing](#ex-1)
- [3. Decision Trees](#3)
    - [Exercise 2 - dt_hyperparams](#ex-2)
- [4. Random Forests](#4)
    - [Exercise 3 - random_forest_grid_search](#ex-3)
- [5. Imputation](#5)
    - [5.1 Error Analysis](#5-1)
        - [Exercise 4 - bad_subset](#ex-4)
    - [5.2 Imputation Approaches](#5-2)
        - [Exercise 5 - hyperparams](#ex-5)
        - [Exercise 6 - hyperparams](#ex-6)
- [6. Comparison](#6)
- [7. Explanations: SHAP](#7)

<a name='1'></a>
## 1. Import Packages

We'll first import all the common packages that we need for this assignment. 

- `shap` is a library that explains predictions made by machine learning models.
- `sklearn` is one of the most popular machine learning libraries.
- `itertools` allows us to conveniently manipulate iterable objects such as lists.
- `pydotplus` is used together with `IPython.display.Image` to visualize graph structures such as decision trees.
- `numpy` is a fundamental package for scientific computing in Python.
- `pandas` is what we'll use to manipulate our data.
- `seaborn` is a plotting library which has some convenient functions for visualizing missing data.
- `matplotlib` is a plotting library.

In [1]:
!pip install -q shap
!pip install -q lifelines

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 575.9/575.9 KB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.7/349.7 KB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 KB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
import os
import sys
import lifelines
import shap
import six
from six import StringIO
import sklearn
import itertools
import pydotplus
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from IPython.display import Image, display 

from sklearn.tree import export_graphviz
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer


In [3]:
#@title helper functions


def cindex(y_true, scores):
    return lifelines.utils.concordance_index(y_true, scores)


def load_data(threshold, data_x, data_y):
    X, y = nhanesi(data_x, data_y)
    df = X.drop([X.columns[0]], axis=1)
    df.loc[:, 'time'] = y
    df.loc[:, 'death'] = np.ones(len(X))
    df.loc[df.time < 0, 'death'] = 0
    df.loc[:, 'time'] = np.abs(df.time)
    df = df.dropna(axis='rows')
    mask = (df.time > threshold) | (df.death == 1)
    df = df[mask]
    X = df.drop(['time', 'death'], axis='columns')
    y = df.time < threshold

    X_dev, X_test, y_dev, y_test = train_test_split(X, y, test_size=0.2, random_state=10)
    feature_y = 'Systolic BP'
    frac = 0.7

    drop_rows = X_dev.sample(frac=frac, replace=False,
                             weights=[prob_drop(X_dev.loc[i, 'Age']) for i in
                                      X_dev.index], random_state=10)
    drop_rows.loc[:, feature_y] = None
    drop_y = y_dev[drop_rows.index]
    X_dev.loc[drop_rows.index, feature_y] = None

    return X_dev, X_test, y_dev, y_test


def prob_drop(age):
    return 1 - (np.exp(0.25 * age - 5) / (1 + np.exp(0.25 * age - 5)))


def nhanesi(data_x, data_y, display=False):
    """Same as shap, but we use local data."""
    X = pd.read_csv(data_x)
    y = pd.read_csv(data_y)["y"]
    if display:
        X_display = X.copy()
        X_display["Sex"] = ["Male" if v == 1 else "Female" for v in X["Sex"]]
        return X_display, np.array(y)
    return X, np.array(y)


In [4]:
#@title tests

### test helper functions

np.random.seed(3)

def datatype_check(expected_output, target_output, error):
    success = 0
    if isinstance(target_output, dict):
        for key in target_output.keys():
            try:
                success += datatype_check(expected_output[key], 
                                         target_output[key], error)
            except:
                print("Error: {} in variable {}. Got {} but expected type {}".format(error,
                                                                          key, type(target_output[key]), type(expected_output[key])))
        if success == len(target_output.keys()):
            return 1
        else:
            return 0
    elif isinstance(target_output, tuple) or isinstance(target_output, list):
        for i in range(len(target_output)):
            try: 
                success += datatype_check(expected_output[i], 
                                         target_output[i], error)
            except:
                print("Error: {} in variable {}, Got {}  but expected type {}".format(error,
                                                                          i, type(target_output[i]), type(expected_output[i])))
        if success == len(target_output):
            return 1
        else:
            return 0
                
    else:
        assert isinstance(target_output, type(expected_output))
        return 1
            
def equation_output_check(expected_output, target_output, error):
    success = 0
    if isinstance(target_output, dict):
        for key in target_output.keys():
            try:
                success += equation_output_check(expected_output[key], 
                                         target_output[key], error)
            except:
                print(expected_output[key], 
                                         target_output[key])
                print("Error: {} for variable {}.".format(error,
                                                                          key))
        if success == len(target_output.keys()):
            return 1
        else:
            return 0
    elif isinstance(target_output, tuple) or isinstance(target_output, list):
        for i in range(len(target_output)):
            try: 
                success += equation_output_check(expected_output[i], 
                                         target_output[i], error)
            except:
                print("Error: {} for variable in position {}.".format(error, i))
        if success == len(target_output):
            return 1
        else:
            return 0
                
    else:
        if hasattr(target_output, 'shape'):
            np.testing.assert_array_almost_equal(target_output, expected_output)
        else:
            assert target_output == expected_output
        return 1
    
def shape_check(expected_output, target_output, error):
    success = 0
    if isinstance(target_output, dict):
        for key in target_output.keys():
            try:
                success += shape_check(expected_output[key], 
                                         target_output[key], error)
            except:
                print("Error: {} for variable {}.".format(error, key))
        if success == len(target_output.keys()):
            return 1
        else:
            return 0
    elif isinstance(target_output, tuple) or isinstance(target_output, list):
        for i in range(len(target_output)):
            try: 
                success += shape_check(expected_output[i], 
                                         target_output[i], error)
            except:
                print("Error: {} for variable {}.".format(error, i))
        if success == len(target_output):
            return 1
        else:
            return 0
                
    else:
        if hasattr(target_output, 'shape'):
            assert target_output.shape == expected_output.shape
        return 1
                
def multiple_test(test_cases, target):
    success = 0
    for test_case in test_cases:
        try:
            target_answer = target(*test_case['input'])
            if test_case['name'] == "datatype_check":
                success += datatype_check(test_case['expected'], target_answer, test_case['error'])
            if test_case['name'] == "equation_output_check":
                success += equation_output_check(test_case['expected'], target_answer, test_case['error'])
            if test_case['name'] == "shape_check":
                success += shape_check(test_case['expected'], target_answer, test_case['error'])
        except:
            print("Error: " + test_case['error'])
            
    if success == len(test_cases):
        print("\033[92m All tests passed.")
    else:
        print('\033[92m', success," Tests passed")
        print('\033[91m', len(test_cases) - success, " Tests failed")
        raise AssertionError("Not all tests were passed for {}. Check your equations and avoid using global variables inside the function.".format(target.__name__))

### general tests

#np.random.seed(3)

### ex1

def fraction_rows_missing_test_case():
    df_test = pd.DataFrame({'a':[None, 1, 1, None], 'b':[1, None, 0, 1]})
    
    return df_test

def fraction_rows_missing_test(target, X_train, X_val, X_test):
    df_test = fraction_rows_missing_test_case()
    
    print("Example dataframe:\n\n", df_test, "\n")
    print("Computed fraction missing: ", target(df_test))
    print("Fraction of rows missing from X_train: ", target(X_train))
    print("Fraction of rows missing from X_val: ", target(X_val))
    print("Fraction of rows missing from X_test: ", target(X_test))
    
    expected_output = 0.75
    
    test_cases = [
        {
            "name":"datatype_check",
            "input": [df_test],
            "expected": expected_output,
            "error": "Data-type mismatch."
        },
        {
            "name": "shape_check",
            "input": [df_test],
            "expected": expected_output,
            "error": "Wrong shape."
        },
        {
            "name": "equation_output_check",
            "input": [df_test],
            "expected": expected_output,
            "error": "Wrong output."
        }
    ]
    
    multiple_test(test_cases, target)


<a name='2'></a>
## 2. The Dataset

Run the next cell to load in the NHANES I epidemiology dataset. This dataset contains various features of hospital patients as well as their outcomes, i.e. whether or not they died within 10 years.

In [5]:
X_dev, X_test, y_dev, y_test = load_data(10, './NHANESI_subset_X.csv', './NHANESI_subset_y.csv')

The dataset has been split into a development set (or dev set), which we will use to develop our risk models, and a test set, which we will use to test our models.

We further split the dev set into a training and validation set, respectively to train and tune our models, using a 75/25 split (note that we set a random state to make this split repeatable).

In [6]:
X_train, X_val, y_train, y_val = train_test_split(X_dev, y_dev, test_size=0.25, random_state=10)

<a name='2-1'></a>
### 2.1 Explore the Dataset

The first step is to familiarize yourself with the data. Run the next cell to get the size of your training set and look at a small sample. 

In [ ]:
print("X_train shape: {}".format(X_train.shape))
X_train.head()

Our targets `y` will be whether or not the target died within 10 years. Run the next cell to see the target data series.

In [ ]:
y_train.head(20)

Use the next cell to examine individual cases and familiarize yourself with the features.

In [ ]:
i = 10
print(X_train.iloc[i,:])
print("\nDied within 10 years? {}".format(y_train.loc[y_train.index[i]]))

<a name='2-2'></a>
### 2.2 Dealing with Missing Data

Looking at our data in `X_train`, we see that some of the data is missing: some values in the output of the previous cell are marked as `NaN` ("not a number").

Missing data is a common occurrence in data analysis, that can be due to a variety of reasons, such as measuring instrument malfunction, respondents not willing or not able to supply information, and errors in the data collection process.

Let's examine the missing data pattern. `seaborn` is an alternative to `matplotlib` that has some convenient plotting functions for data analysis. We can use its `heatmap` function to easily visualize the missing data pattern.

Run the cell below to plot the missing data: 

In [ ]:
sns.heatmap(X_train.isnull(), cbar=False)
plt.title("Training")
plt.show()

sns.heatmap(X_val.isnull(), cbar=False)
plt.title("Validation")
plt.show()

For each feature, represented as a column, values that are present are shown in black, and missing values are set in a light color.

From this plot, we can see that many values are missing for systolic blood pressure (`Systolic BP`).


<a name='ex-1'></a>
### Exercise 1 - fraction_rows_missing

In the cell below, write a function to compute the fraction of cases with missing data. This will help us decide how we handle this missing data in the future.

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<p>
<ul>
    <li> The <code>pandas.DataFrame.isnull()</code> method is helpful in this case.</li>
    <li> Use the <code>pandas.DataFrame.any()</code> method and set the <code>axis</code> parameter.</li>
    <li> Divide the total number of rows with missing data by the total number of rows. Remember that in Python, <code>True</code> values are equal to 1.</li>
</ul>
</p>

In [11]:
def fraction_rows_missing(df):
    '''
    Return percent of rows with any missing
    data in the dataframe. 
    
    Input:
        df (dataframe): a pandas dataframe with potentially missing data
    Output:
        frac_missing (float): fraction of rows with missing data
    '''
    return sum(df.isnull().any(axis=1)) / len(df)

In [ ]:
### test cell ex1 - do not modify this test cell   
fraction_rows_missing_test(fraction_rows_missing, X_train, X_val, X_test)

#### Expected Output:
```
Computed fraction missing:  0.75
Fraction of rows missing from X_train:  0.6986594132504371
Fraction of rows missing from X_val:  0.703962703962704
Fraction of rows missing from X_test:  0.0
 All tests passed.
``` 

We see that our train and validation sets have missing values, but luckily our test set has complete cases.

As a first pass, we will begin with a **complete case analysis**, dropping all of the rows with any missing data. Run the following cell to drop these rows from our train and validation sets. 

In [ ]:
X_train_dropped = X_train.dropna(axis='rows')
y_train_dropped = y_train.loc[X_train_dropped.index]
X_val_dropped = X_val.dropna(axis='rows')
y_val_dropped = y_val.loc[X_val_dropped.index]

### Notice the new shape of X
print("X_train_dropped shape: {}".format(X_train_dropped.shape))
X_train_dropped.head() 

<a name='3'></a>
## 3. Decision Trees

Having just learned about decision trees, you choose to use a decision tree classifier. Use scikit-learn to build a decision tree for the hospital dataset using the train set.

In [ ]:
dt = DecisionTreeClassifier(max_depth=None, random_state=10)
dt.fit(X_train_dropped, y_train_dropped)

Next we will evaluate our model. We'll use C-Index for evaluation.

> Remember from lesson 4 of week 1 that the C-Index evaluates the ability of a model to differentiate between different classes, by quantifying how often, when considering all pairs of patients (A, B), the model says that patient A has a higher risk score than patient B when, in the observed data, patient A actually died and patient B actually lived. In our case, our model is a binary classifier, where each risk score is either 1 (the model predicts that the patient will die) or 0 (the patient will live).
>
> More formally, defining _permissible pairs_ of patients as pairs where the outcomes are different, _concordant pairs_ as permissible pairs where the patient that died had a higher risk score (i.e. our model predicted 1 for the patient that died and 0 for the one that lived), and _ties_ as permissible pairs where the risk scores were equal (i.e. our model predicted 1 for both patients or 0 for both patients), the C-Index is equal to:
>
> $$\text{C-Index} = \frac{\#\text{concordant pairs} + 0.5\times \#\text{ties}}{\#\text{permissible pairs}}$$

Run the next cell to compute the C-Index on the train and validation set (we've given you an implementation this time).

In [ ]:
y_train_preds = dt.predict_proba(X_train_dropped)[:, 1]
print(f"Train C-Index: {cindex(y_train_dropped.values, y_train_preds)}")


y_val_preds = dt.predict_proba(X_val_dropped)[:, 1]
print(f"Val C-Index: {cindex(y_val_dropped.values, y_val_preds)}")

Unfortunately your tree seems to be overfitting: it fits the training data so closely that it doesn't generalize well to other samples such as those from the validation set.

> The training C-index comes out to 1.0 because, when initializing `DecisionTreeClasifier`, we have left `max_depth` and `min_samples_split` unspecified. The resulting decision tree will therefore keep splitting as far as it can, which pretty much guarantees a pure fit to the training data.

To handle this, you can change some of the hyperparameters of our tree.

<a name='ex-2'></a>
### Exercise 2 - dt_hyperparams

Try and find a set of hyperparameters that improves the generalization to the validation set and recompute the C-index. If you do it right, you should get C-index above 0.6 for the validation set. 

You can refer to the documentation for the sklearn [DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html).

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<p>
<ul>
    <li> Try limiting the depth of the tree (max_depth).</li>
</ul>
</p>

### Experiment with different hyperparameters for the DecisionTreeClassifier
### until you get a c-index above 0.6 for the validation set
dt_hyperparams = {
    # set your own hyperparameters below, such as 'min_samples_split': 1
    
    'None': None
    
}


Run the next cell to fit and evaluate the regularized tree.

In [ ]:
dt_hyperparams = {
    # set your own hyperparameters below, such as 'min_samples_split': 1
    'max_depth': 3,
}

dt_reg = DecisionTreeClassifier(**dt_hyperparams, random_state=10)
dt_reg.fit(X_train_dropped, y_train_dropped)

y_train_preds = dt_reg.predict_proba(X_train_dropped)[:, 1]
y_val_preds = dt_reg.predict_proba(X_val_dropped)[:, 1]
print(f"Train C-Index: {cindex(y_train_dropped.values, y_train_preds)}")
print(f"Val C-Index (expected > 0.6): {cindex(y_val_dropped.values, y_val_preds)}")

#### Expected Output:
```
Train C-Index > 0.6
Val C-Index > 0.6
```

If your output is not greater than `0.6`, try changing and tweaking your hyperparameters in `Ex 2`.

If you used a low `max_depth` you can print the entire tree. This allows for easy interpretability. Run the next cell to print the tree splits. 

In [ ]:
dot_data = StringIO()
export_graphviz(dt_reg, feature_names=X_train_dropped.columns, out_file=dot_data,  
                filled=True, rounded=True, proportion=True, special_characters=True,
                impurity=False, class_names=['neg', 'pos'], precision=2)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())

> **Overfitting, underfitting, and the bias-variance tradeoff**
>
> If you tested several values of `max_depth`, you may have seen that a value of `3` gives training and validation C-Indices of about `0.689` and `0.630`, and that a `max_depth` of `2` gives better agreement with values of about `0.653` and `0.607`. In the latter case, we have further reduced overfitting, at the cost of a minor loss in predictive performance.
>
> Contrast this with a `max_depth` value of `1`, which results in C-Indices of about `0.597` for the training set and `0.598` for the validation set: we have eliminated overfitting but with a much stronger degradation of predictive performance.
>
> Lower predictive performance on the training and validation sets is indicative of the model _underfitting_ the data: it neither learns enough from the training data nor is able to generalize to unseen data (the validation data in our case).
>
> Finding a model that minimizes and acceptably balances underfitting and overfitting (e.g. selecting the model with a `max_depth` of `2` over the other values) is a common problem in machine learning that is known as the _bias-variance tradeoff_.

<a name='4'></a>
## 4. Random Forests

No matter how you choose hyperparameters, a single decision tree is prone to overfitting. To solve this problem, you can try **random forests**, which combine predictions from many different trees to create a robust classifier. 

As before, we will use scikit-learn to build a random forest for the data. We will use the default hyperparameters.

In [ ]:
rf = RandomForestClassifier(n_estimators=100, random_state=10)
rf.fit(X_train_dropped, y_train_dropped)

Now compute and report the C-Index for the random forest on the training and validation set.

In [19]:
y_train_rf_preds = rf.predict_proba(X_train_dropped)[:, 1]
print(f"Train C-Index: {cindex(y_train_dropped.values, y_train_rf_preds)}")

y_val_rf_preds = rf.predict_proba(X_val_dropped)[:, 1]
print(f"Val C-Index: {cindex(y_val_dropped.values, y_val_rf_preds)}")

Train C-Index: 1.0
Val C-Index: 0.6660488696808511


Training a random forest with the default hyperparameters results in a model that has better predictive performance than individual decision trees as in the previous section, but this model is overfitting.

We therefore need to tune (or optimize) the hyperparameters, to find a model that both has good predictive performance and minimizes overfitting.

The hyperparameters we choose to adjust will be:

- `n_estimators`: the number of trees used in the forest.
- `max_depth`: the maximum depth of each tree.
- `min_samples_leaf`: the minimum number (if `int`) or proportion (if `float`) of samples in a leaf.

The approach we implement to tune the hyperparameters is known as a grid search:

- We define a set of possible values for each of the target hyperparameters.

- A model is trained and evaluated for every possible combination of hyperparameters.

- The best performing set of hyperparameters is returned.

The cell below implements a hyperparameter grid search, using the C-Index to evaluate each tested model.

In [20]:
def holdout_grid_search(clf, X_train_hp, y_train_hp, X_val_hp, y_val_hp, hyperparams, fixed_hyperparams={}):
    '''
    Conduct hyperparameter grid search on hold out validation set. Use holdout validation.
    Hyperparameters are input as a dictionary mapping each hyperparameter name to the
    range of values they should iterate over. Use the cindex function as your evaluation
    function.

    Input:
        clf: sklearn classifier
        X_train_hp (dataframe): dataframe for training set input variables
        y_train_hp (dataframe): dataframe for training set targets
        X_val_hp (dataframe): dataframe for validation set input variables
        y_val_hp (dataframe): dataframe for validation set targets
        hyperparams (dict): hyperparameter dictionary mapping hyperparameter
                            names to range of values for grid search
        fixed_hyperparams (dict): dictionary of fixed hyperparameters that
                                  are not included in the grid search

    Output:
        best_estimator (sklearn classifier): fitted sklearn classifier with best performance on
                                             validation set
        best_hyperparams (dict): hyperparameter dictionary mapping hyperparameter
                                 names to values in best_estimator
    '''
    best_estimator = None
    best_hyperparams = {}
    
    # hold best running score
    best_score = 0.0

    # get list of param values
    lists = hyperparams.values()
    
    # get all param combinations
    param_combinations = list(itertools.product(*lists))
    total_param_combinations = len(param_combinations)

    # iterate through param combinations
    for i, params in enumerate(param_combinations, 1):
        # fill param dict with params
        param_dict = {}
        for param_index, param_name in enumerate(hyperparams):
            param_dict[param_name] = params[param_index]
            
        # create estimator with specified params
        estimator = clf(**param_dict, **fixed_hyperparams)

        # fit estimator
        estimator.fit(X_train_hp, y_train_hp)
        
        # get predictions on validation set
        preds = estimator.predict_proba(X_val_hp)
        
        # compute cindex for predictions
        estimator_score = cindex(y_val_hp, preds[:,1])

        print(f'[{i}/{total_param_combinations}] {param_dict}')
        print(f'Val C-Index: {estimator_score}\n')

        # if new high score, update high score, best estimator
        # and best params 
        if estimator_score >= best_score:
                best_score = estimator_score
                best_estimator = estimator
                best_hyperparams = param_dict

    # add fixed hyperparamters to best combination of variable hyperparameters
    best_hyperparams.update(fixed_hyperparams)
    
    return best_estimator, best_hyperparams

<a name='ex-3'></a>
### Exercise 3 - random_forest_grid_search

In the cell below, define the values you want to run the hyperparameter grid search on, and run the cell to find the best-performing set of hyperparameters.

Your objective is to get a C-Index above `0.6` on both the train and validation set.

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<p>
<ul>
    <li>n_estimators: try values greater than 100</li>
    <li>max_depth: try values in the range 1 to 100</li>
    <li>min_samples_leaf: try float values below .5 and/or int values greater than 2</li>
</ul>
</p>

In [21]:
def random_forest_grid_search(X_train_dropped, y_train_dropped, X_val_dropped, y_val_dropped):

    # Define ranges for the chosen random forest hyperparameters 
    hyperparams = {
        # how many trees should be in the forest (int)
        'n_estimators': [1],

        # the maximum depth of trees in the forest (int)
        
        'max_depth': [3, 4],
        
        # the minimum number of samples in a leaf as a fraction
        # of the total number of samples in the training set
        # Can be int (in which case that is the minimum number)
        # or float (in which case the minimum is that fraction of the
        # number of training set samples)
        'min_samples_leaf': [1],
    }

    
    fixed_hyperparams = {
        'random_state': 10,
    }
    
    rf = RandomForestClassifier

    best_rf, best_hyperparams = holdout_grid_search(rf, X_train_dropped, y_train_dropped,
                                                    X_val_dropped, y_val_dropped, hyperparams,
                                                    fixed_hyperparams)

    print(f"Best hyperparameters:\n{best_hyperparams}")

    
    y_train_best = best_rf.predict_proba(X_train_dropped)[:, 1]
    print(f"Train C-Index: {cindex(y_train_dropped, y_train_best)}")

    y_val_best = best_rf.predict_proba(X_val_dropped)[:, 1]
    print(f"Val C-Index: {cindex(y_val_dropped, y_val_best)}")
    
    # add fixed hyperparamters to best combination of variable hyperparameters
    best_hyperparams.update(fixed_hyperparams)
    
    return best_rf, best_hyperparams

In [ ]:
best_rf, best_hyperparams = random_forest_grid_search(X_train_dropped, y_train_dropped, X_val_dropped, y_val_dropped)

Finally, evaluate the model on the test set. This is a crucial step, as trying out many combinations of hyperparameters and evaluating them on the validation set could result in a model that ends up overfitting the validation set. We therefore need to check if the model performs well on unseen data, which is the role of the test set, which we have held out until now.

In [23]:

y_test_best = best_rf.predict_proba(X_test)[:, 1]

print(f"Test C-Index: {cindex(y_test.values, y_test_best)}")

Test C-Index: 0.623706805620016


#### Expected Output:
```
Test C-Index > 0.6
```

If your output is not greater than `0.6`, try changing and tweaking your hyperparameters in `Ex 3`.

<a name='5'></a>
## 5. Imputation

You've now built and optimized a random forest model on our data. However, there was still a drop in test C-Index. This might be because you threw away more than half of the data of our data because of missing values for systolic blood pressure. Instead, we can try filling in, or imputing, these values. 

First, let's explore to see if our data is missing at random or not. Let's plot histograms of the dropped rows against each of the covariates (aside from systolic blood pressure) to see if there is a trend. Compare these to the histograms of the feature in the entire dataset. Try to see if one of the covariates has a signficantly different distribution in the two subsets.

In [ ]:
dropped_rows = X_train[X_train.isnull().any(axis=1)]

columns_except_Systolic_BP = [col for col in X_train.columns if col not in ['Systolic BP']]

for col in columns_except_Systolic_BP:
    sns.distplot(X_train.loc[:, col], norm_hist=True, kde=False, label='full data')
    sns.distplot(dropped_rows.loc[:, col], norm_hist=True, kde=False, label='without missing data')
    plt.legend()

    plt.show()

Most of the covariates are distributed similarly whether or not we have discarded rows with missing data. In other words missingness of the data is independent of these covariates.

If this had been true across *all* covariates, then the data would have been said to be **missing completely at random (MCAR)**.

But when considering the age covariate, we see that much more data tends to be missing for patients over 65. The reason could be that blood pressure was measured less frequently for old people to avoid placing additional burden on them.

As missingness is related to one or more covariates, the missing data is said to be **missing at random (MAR)**.

Based on the information we have, there is however no reason to believe that the _values_ of the missing data — or specifically the values of the missing systolic blood pressures — are related to the age of the patients. 
If this was the case, then this data would be said to be **missing not at random (MNAR)**.

<a name='5-1'></a>
### 5.1 Error Analysis

<a name='ex-4'></a>
### Exercise 4 - bad_subset
Using the information from the plots above, try to find a subgroup of the test data on which the model performs poorly. You should be able to easily find a subgroup of at least 250 cases on which the model has a C-Index of less than 0.69.

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<p>
<ul>
    <li> Define a mask using a feature and a threshold, e.g. patients with a BMI below 20: <code>mask = X_test['BMI'] < 20 </code>. </li>
    <li> Try to find a subgroup for which the model had little data.</li>
</ul>
</p>

In [25]:

def bad_subset(forest, X_test, y_test):
    # define mask to select large subset with poor performance
    # currently mask defines the entire set
    
    mask = X_test.Age < 40

    X_subgroup = X_test[mask]
    y_subgroup = y_test[mask]
    subgroup_size = len(X_subgroup)

    y_subgroup_preds = forest.predict_proba(X_subgroup)[:, 1]
    performance = cindex(y_subgroup.values, y_subgroup_preds)
    
    return performance, subgroup_size

In [ ]:
#### Test Your Work
performance, subgroup_size = bad_subset(best_rf, X_test, y_test)
print("Subgroup size should greater than 250, performance should be less than 0.69")
print(f"Your Subgroup size: {subgroup_size}, and your C-Index: {performance}")

#### Expected Output
Note, your actual output will vary depending on the hyperparameters and the mask that you chose.

```Python
Your Subgroup size > 250, and your C-Index < 0.69
```

<a name='5-2'></a>
### 5.2 Imputation Approaches

Seeing that our data is not missing completely at random, we can handle the missing values by replacing them with substituted values based on the other values that we have. This is known as imputation.

The first imputation strategy that we will use is **mean substitution**: we will replace the missing values for each feature with the mean of the available values. In the next cell, use the `SimpleImputer` from `sklearn` to use mean imputation for the missing values.

In [27]:
# Impute values using the mean
imputer = SimpleImputer(strategy='mean')
imputer.fit(X_train)
X_train_mean_imputed = pd.DataFrame(imputer.transform(X_train), columns=X_train.columns)
X_val_mean_imputed = pd.DataFrame(imputer.transform(X_val), columns=X_val.columns)

<a name='ex-5'></a>
### Exercise 5 - hyperparams
Now perform a hyperparameter grid search to find the best-performing random forest model, and report results on the test set. 

Define the parameter ranges for the hyperparameter search in the next cell, and run the cell.

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<p>
<ul>
    <li>n_estimators: try values greater than 100</li>
    <li>max_depth: try values in the range 1 to 100</li>
    <li>min_samples_leaf: try float values below .5 and/or int values greater than 2</li>
</ul>
</p>


In [ ]:
# Define ranges for the random forest hyperparameter search 
hyperparams = {

    # how many trees should be in the forest (int)
    'n_estimators': [1],

    # the maximum depth of trees in the forest (int)
    'max_depth': [3, 4],

    # the minimum number of samples in a leaf as a fraction
    # of the total number of samples in the training set
    # Can be int (in which case that is the minimum number)
    # or float (in which case the minimum is that fraction of the
    # number of training set samples)
    'min_samples_leaf': [1],
}

In [34]:

rf = RandomForestClassifier

rf_mean_imputed, best_hyperparams_mean_imputed = holdout_grid_search(rf, X_train_mean_imputed, y_train,
                                                                     X_val_mean_imputed, y_val,
                                                                     hyperparams, {'random_state': 10})

print("Performance for best hyperparameters:")

y_train_best = rf_mean_imputed.predict_proba(X_train_mean_imputed)[:, 1]
print(f"- Train C-Index: {cindex(y_train, y_train_best):.4f}")

y_val_best = rf_mean_imputed.predict_proba(X_val_mean_imputed)[:, 1]
print(f"- Val C-Index: {cindex(y_val, y_val_best):.4f}")

y_test_imp = rf_mean_imputed.predict_proba(X_test)[:, 1]
print(f"- Test C-Index: {cindex(y_test, y_test_imp):.4f}")

[1/2] {'n_estimators': 1, 'max_depth': 3, 'min_samples_leaf': 1}
Val C-Index: 0.7065655677369344

[2/2] {'n_estimators': 1, 'max_depth': 4, 'min_samples_leaf': 1}
Val C-Index: 0.6940421983805932

Performance for best hyperparameters:
- Train C-Index: 0.7229
- Val C-Index: 0.7066
- Test C-Index: 0.7168


#### Expected output
Note, your actual C-Index values will vary depending on the hyperparameters that you choose.

```
C-Index >= 0.74
```

- Try to get a good C-Index, similar these numbers below:

```Python
Performance for best hyperparameters:
- Train C-Index: 0.8109
- Val C-Index: 0.7495
- Test C-Index: 0.7805
```

Next, we will apply another imputation strategy, known as **multivariate feature imputation**, using scikit-learn's `IterativeImputer` class (see the [documentation](https://scikit-learn.org/stable/modules/impute.html#iterative-imputer)).

With this strategy, for each feature that is missing values, a regression model is trained to predict observed values based on all of the other features, and the missing values are inferred using this model.
As a single iteration across all features may not be enough to impute all missing values, several iterations may be performed, hence the name of the class `IterativeImputer`.

In the next cell, use `IterativeImputer` to perform multivariate feature imputation.

> Note that the first time the cell is run, `imputer.fit(X_train)` may fail with the message `LinAlgError: SVD did not converge`: simply re-run the cell.

In [35]:
# Impute using regression on other covariates
imputer = IterativeImputer(random_state=0, sample_posterior=False, max_iter=1, min_value=0)
imputer.fit(X_train)
X_train_imputed = pd.DataFrame(imputer.transform(X_train), columns=X_train.columns)
X_val_imputed = pd.DataFrame(imputer.transform(X_val), columns=X_val.columns)

[IterativeImputer] Early stopping criterion not reached.


<a name='ex-6'></a>
### Exercise 6 - hyperparams

Perform a hyperparameter grid search to find the best-performing random forest model, and report results on the test set. Define the parameter ranges for the hyperparameter search in the next cell, and run the cell.

#### Target performance

Try to get a text c-index of at least 0.74 or higher.

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<p>
<ul>
    <li>n_estimators: try values greater than 100</li>
    <li>max_depth: try values in the range 1 to 100</li>
    <li>min_samples_leaf: try float values below .5 and/or int values greater than 2</li>
</ul>
</p>


In [36]:
# Define ranges for the random forest hyperparameter search 
hyperparams = {

    # how many trees should be in the forest (int)
    'n_estimators': [1],

    # the maximum depth of trees in the forest (int)
    'max_depth': [3, 4],

    # the minimum number of samples in a leaf as a fraction
    # of the total number of samples in the training set
    # Can be int (in which case that is the minimum number)
    # or float (in which case the minimum is that fraction of the
    # number of training set samples)
    'min_samples_leaf': [1],
}

In [37]:
rf = RandomForestClassifier

rf_imputed, best_hyperparams_imputed = holdout_grid_search(rf, X_train_imputed, y_train,
                                                           X_val_imputed, y_val,
                                                           hyperparams, {'random_state': 10})

print("Performance for best hyperparameters:")

y_train_best = rf_imputed.predict_proba(X_train_imputed)[:, 1]
print(f"- Train C-Index: {cindex(y_train, y_train_best):.4f}")

y_val_best = rf_imputed.predict_proba(X_val_imputed)[:, 1]
print(f"- Val C-Index: {cindex(y_val, y_val_best):.4f}")

y_test_imp = rf_imputed.predict_proba(X_test)[:, 1]
print(f"- Test C-Index: {cindex(y_test, y_test_imp):.4f}")

[1/2] {'n_estimators': 1, 'max_depth': 3, 'min_samples_leaf': 1}
Val C-Index: 0.7081332959857906

[2/2] {'n_estimators': 1, 'max_depth': 4, 'min_samples_leaf': 1}
Val C-Index: 0.6958227207684908

Performance for best hyperparameters:
- Train C-Index: 0.7239
- Val C-Index: 0.7081
- Test C-Index: 0.7357


#### Expected Output
Note, your actual C-Index values will vary depending on the hyperparameters that you choose.

```
C-Index >= 0.74
```

- Try to get a good C-Index, similar these numbers below:

```Python
Performance for best hyperparameters:
- Train C-Index: 0.8131
- Val C-Index: 0.7454
- Test C-Index: 0.7797
```

<a name='6'></a>
## 6. Comparison

For good measure, retest on the subgroup from before to see if your new models do better.

In [38]:
performance, subgroup_size = bad_subset(best_rf, X_test, y_test)
print(f"C-Index (no imputation): {performance}")

performance, subgroup_size = bad_subset(rf_mean_imputed, X_test, y_test)
print(f"C-Index (mean imputation): {performance}")

performance, subgroup_size = bad_subset(rf_imputed, X_test, y_test)
print(f"C-Index (multivariate feature imputation): {performance}")

C-Index (no imputation): 0.5032160963244613
C-Index (mean imputation): 0.5648922686945501
C-Index (multivariate feature imputation): 0.5613434727503168


We should see that avoiding complete case analysis (i.e. analysis only on observations for which there is no missing data) allows our model to generalize a bit better. Remember to examine your missing cases to judge whether they are missing at random or not!

<a name='7'></a>
## 7. Explanations: SHAP

Using a random forest has improved results, but we've lost some of the natural interpretability of trees. In this section we'll try to explain the predictions using slightly more sophisticated techniques. 

You choose to apply **SHAP (SHapley Additive exPlanations)**, a cutting edge method that explains predictions made by black-box machine learning models (i.e. models which are too complex to be understandable by humans as is).

> Given a prediction made by a machine learning model, SHAP values explain the prediction by quantifying the additive importance of each feature to the prediction. SHAP values have their roots in cooperative game theory, where Shapley values are used to quantify the contribution of each player to the game.
> 
> Although it is computationally expensive to compute SHAP values for general black-box models, in the case of trees and forests there exists a fast polynomial-time algorithm. For more details, see the [TreeShap paper](https://arxiv.org/pdf/1802.03888.pdf).

We'll use the [shap library](https://github.com/slundberg/shap) to do this for our random forest model. Run the next cell to output the most at risk individuals in the test set according to our model.

In [ ]:
X_test_risk = X_test.copy(deep=True)
X_test_risk.loc[:, 'risk'] = rf_imputed.predict_proba(X_test_risk)[:, 1]
X_test_risk = X_test_risk.sort_values(by='risk', ascending=False)
X_test_risk.head()

We can use SHAP values to try and understand the model output on specific individuals using force plots. Run the cell below to see a force plot on the riskiest individual. 

In [ ]:
explainer = shap.TreeExplainer(rf_imputed)
i = 0
shap_value = explainer.shap_values(X_test.loc[X_test_risk.index[i], :])[1]
shap.force_plot(explainer.expected_value[1], shap_value, feature_names=X_test.columns, matplotlib=True)

How to read this chart:
- The red sections on the left are features which push the model towards the final prediction in the positive direction (i.e. a higher Age increases the predicted risk).
- The blue sections on the right are features that push the model towards the final prediction in the negative direction (if an increase in a feature leads to a lower risk, it will be shown in blue).
- Note that the exact output of your chart will differ depending on the hyper-parameters that you choose for your model.

We can also use SHAP values to understand the model output in aggregate. Run the next cell to initialize the SHAP values (this may take a few minutes).

In [41]:
shap_values = shap.TreeExplainer(rf_imputed).shap_values(X_test)[1]

Run the next cell to see a summary plot of the SHAP values for each feature on each of the test examples. The colors indicate the value of the feature.

In [ ]:
shap.summary_plot(shap_values, X_test)

Clearly we see that being a woman (`sex = 2.0`, as opposed to men for which `sex = 1.0`) has a negative SHAP value, meaning that it reduces the risk of dying within 10 years. High age and high systolic blood pressure have positive SHAP values, and are therefore related to increased mortality. 

You can see how features interact using dependence plots. These plot the SHAP value for a given feature for each data point, and color the points in using the value for another feature. This lets us begin to explain the variation in SHAP value for a single value of the main feature.

Run the next cell to see the interaction between Age and Sex.

In [ ]:
shap.dependence_plot('Age', shap_values, X_test, interaction_index='Sex')

We see that while Age > 50 is generally bad (positive SHAP value), being a woman generally reduces the impact of age. This makes sense since we know that women generally live longer than men.

Let's now look at poverty index and age.

In [ ]:
shap.dependence_plot('Poverty index', shap_values, X_test, interaction_index='Age')

We see that the impact of poverty index drops off quickly, and for higher income individuals age begins to explain much of variation in the impact of poverty index.

Try some other pairs and see what other interesting relationships you can find!

<a name='8'></a>
## 7. Feature Importance - Permutations and shapley values in more detail

When developing predictive models and risk measures, it's often helpful to know which features are making the most difference. This is easy to determine in simpler models such as linear models and decision trees. However as we move to more complex models to achieve high performance, we usually sacrifice some interpretability. In this assignment we'll try to regain some of that interpretability using Shapley values, a technique which has gained popularity in recent years, but which is based on classic results in cooperative game theory.

In [45]:
#@title let's test our model

def cindex(y_true, scores):
    return lifelines.utils.concordance_index(y_true, scores)

### loading model if saved vs just use our rf_mean_imputed from above to test it
## rf = pickle.load(open('./whatever_model_we have', 'rb'))

test_df = pd.read_csv('./nhanest_test.csv')
test_df = test_df.drop(test_df.columns[0], axis=1)
X_test = test_df.drop('y', axis=1)
y_test = test_df.loc[:, 'y']
cindex_test = cindex(y_test, rf_mean_imputed.predict_proba(X_test)[:, 1])

print("Model C-index on test: {}".format(cindex_test))

Model C-index on test: 0.7168262942491878


In [ ]:
## Run the next cell to print out the riskiest individuals according to our model
X_test_risky = X_test.copy(deep=True)
X_test_risky.loc[:, 'risk'] = rf_mean_imputed.predict_proba(X_test)[:, 1] # Predicting our risk.
X_test_risky = X_test_risky.sort_values(by='risk', ascending=False) # Sorting by risk value.
X_test_risky.head()

In [ ]:
# Permuation Method for Feature Importance
# First we'll try to determine feature importance using the permutation method. 
#In the permutation method, the importance of feature i would be the regular performance 
#of the model minus the performance with the values for feature i permuted in the dataset. 
#This way we can assess how well a model without that feature would do without having to train a new model for each feature.
# Implementing Permutation - permute_feature
# Complete the implementation of the function below, which given a feature name returns a 
# dataset with those feature values randomly permuted.

def permute_feature(df, feature):
    """
    Given dataset, returns version with the values of
    the given feature randomly permuted. 

    Args:
        df (dataframe): The dataset, shape (num subjects, num features)
        feature (string): Name of feature to permute
    Returns:
        permuted_df (dataframe): Exactly the same as df except the values
                                of the given feature are randomly permuted.
    """
    permuted_df = df.copy(deep=True) # Make copy so we don't change original df

    # Permute the values of the column 'feature'
    permuted_features = np.random.permutation(permuted_df[feature])
    
    # Set the column 'feature' to its permuted values.
    permuted_df[feature] = permuted_features

    return permuted_df


def permutation_importance(X, y, model, metric, num_samples = 100):
    """
    Compute permutation importance for each feature.

    Args:
        X (dataframe): Dataframe for test data, shape (num subject, num features)
        y (np.array): Labels for each row of X, shape (num subjects,)
        model (object): Model to compute importances for, guaranteed to have
                        a 'predict_proba' method to compute probabilistic 
                        predictions given input
        metric (function): Metric to be used for feature importance. Takes in ground
                           truth and predictions as the only two arguments
        num_samples (int): Number of samples to average over when computing change in
                           performance for each feature
    Returns:
        importances (dataframe): Dataframe containing feature importance for each
                                 column of df with shape (1, num_features)
    """

    importances = pd.DataFrame(index = ['importance'], columns = X.columns)
    
    # Get baseline performance (note, you'll use this metric function again later)
    baseline_performance = metric(y, model.predict_proba(X)[:, 1])

    # Iterate over features (the columns in the importances dataframe)
    for feature in importances.columns: # complete this line
        
        # Compute 'num_sample' performances by permutating that feature
        
        # You'll see how the model performs when the feature is permuted
        # You'll do this num_samples number of times, and save the performance each time
        # To store the feature performance,
        # create a numpy array of size num_samples, initialized to all zeros
        feature_performance_arr = np.zeros(num_samples)
        
        # Loop through each sample
        for i in range(num_samples): # complete this line
            
            # permute the column of dataframe X
            perm_X = permute_feature(X,feature)
            
            # calculate the performance with the permuted data
            # Use the same metric function that was used earlier
            feature_performance_arr[i] = metric(y, model.predict_proba(perm_X)[:, 1])
    
        # Compute importance: absolute difference between 
        # the baseline performance and the average across the feature performance
        importances[feature]['importance'] = baseline_performance - np.mean(feature_performance_arr)

    return importances

importances = permutation_importance(X_test, y_test, rf_mean_imputed, cindex, num_samples=100)
importances

In [ ]:
# let's plot for better understanding
importances.T.plot.bar()
plt.ylabel("Importance")
l = plt.legend()
l.remove()
plt.show()

We see age as by far the best prediction of near term mortality, as one might expect. Next is sex, followed by diastolic blood pressure. Interestingly, the poverty index also has a large impact, despite the fact that it is not directly related to an individual's health. This alludes to the importance of social determinants of health in our model.


**Closer look on Shapley Values for Random Forests**

We'll contrast the permutation method with a more recent technique known as Shapley values (actually, Shapley values date back to the mid 20th century, but have only been applied to machine learning very recently).


Visualizing Feature Importance on Specific Individuals
We can use Shapley values to try and understand the model output on specific individuals. In general Shapley values take exponential time to compute, but luckily there are faster approximations for forests in particular that run in polynomial time. Run the next cell to display a 'force plot' showing how each feature influences the output for the first person in our dataset. If you want more information about 'force plots' and other decision plots, please take a look at GitHub repo by the shap library creators.

In [ ]:
explainer = shap.TreeExplainer(rf_mean_imputed)
i = 0 # Picking an individual
shap_value = explainer.shap_values(X_test.loc[X_test_risky.index[i], :])[1]
shap.force_plot(explainer.expected_value[1], shap_value, feature_names=X_test.columns, matplotlib=True)

For this individual, their age, pulse pressure, and sex were the biggest contributors to their high risk prediction. Note how shapley values give us greater granularity in our interpretations.

Feel free to change the i value above to explore the feature influences for different individuals.

**Visualizing Feature Importance on Aggregate**

Just like with the permutation method, we might also want to understand model output in aggregate. Shapley values allow us to do this as well. Run the next cell to initialize the shapley values for each example in the test set (this may also take a few minutes).

In [53]:
shap_values = shap.TreeExplainer(rf_mean_imputed).shap_values(X_test)[1]

Run the next cell to see a summary plot of the shapley values for each feature on each of the test examples. The colors indicate the value of the feature. The features are listed in terms of decreasing absolute average shapley value over all the individuals in the dataset.

In the plot, you might be able to notice a high concentration of points on specific SHAP value ranges. This means that a high proportion of our test set lies on those ranges.

As with the permutation method, age, sex, poverty index, and diastolic BP seem to be the most important features. Being older has a negative impact on mortality, and being a woman (sex=2.0) has a positive effect.

In [ ]:
shap.summary_plot(shap_values, X_test)

**Visualizing Interactions between Features**

The shap library also lets you visualize interactions between features using dependence plots. These plot the Shapley value for a given feature for each data point, and color the points in using the value for another feature. This lets us begin to explain the variation in shapley value for a single value of the main feature.

Run the next cell to see the interaction between Age and Sex.

shap.dependence_plot('Age', shap_values, X_test, interaction_index = 'Sex')
We see that while Age > 50 is generally bad (positive Shapley value), being a woman (red points) generally reduces the impact of age. This makes sense since we know that women generally live longer than men.

In [ ]:
shap.dependence_plot('Age', shap_values, X_test, interaction_index = 'Sex')
shap.dependence_plot('Poverty index', shap_values, X_test, interaction_index='Age')

# **B.** Survival Estimates that Vary with Time

Welcome to the third assignment of Course 2. In this assignment, we'll use Python to build some of the statistical models we learned this past week to analyze surivival estimates for a dataset of lymphoma patients. We'll also evaluate these models and interpret their outputs. Along the way, you will be learning about the following: 

- Censored Data
- Kaplan-Meier Estimates
- Subgroup Analysis

## Table of Contents

- [1. Import Packages](#1)
- [2. Load the Dataset](#2)
- [3. Censored Data](#3)
    - [Exercise 1 - frac_censored](#ex-1)
- [4. Survival Estimates](#4)
    - [Exercise 2 - naive_estimator](#ex-2)
    - [Exercise 3 - HomemadeKM](#ex-3)
- [5. Subgroup Analysis](#5)
    - [5.1 Bonus: Log Rank Test](#5-1)

<a name='1'></a>
## 1. Import Packages

We'll first import all the packages that we need for this assignment. 

- `lifelines` is an open-source library for data analysis.
- `numpy` is the fundamental package for scientific computing in python.
- `pandas` is what we'll use to manipulate our data.
- `matplotlib` is a plotting library.

In [ ]:
!pip install -q lifelines

In [ ]:
import sys
from IPython.display import display
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import lifelines
from lifelines import KaplanMeierFitter as KM
from lifelines.statistics import logrank_test
from lifelines.datasets import load_lymphoma

In [ ]:
#@title tests

### data tests
np.random.seed(3)

def datatype_check(expected_output, target_output, error):
    success = 0
    if isinstance(target_output, dict):
        for key in target_output.keys():
            try:
                success += datatype_check(expected_output[key], 
                                         target_output[key], error)
            except:
                print("Error: {} in variable {}. Got {} but expected type {}".format(error,
                                                                          key, type(target_output[key]), type(expected_output[key])))
        if success == len(target_output.keys()):
            return 1
        else:
            return 0
    elif isinstance(target_output, tuple) or isinstance(target_output, list):
        for i in range(len(target_output)):
            try: 
                success += datatype_check(expected_output[i], 
                                         target_output[i], error)
            except:
                print("Error: {} in variable {}, Got {}  but expected type {}".format(error,
                                                                          i, type(target_output[i]), type(expected_output[i])))
        if success == len(target_output):
            return 1
        else:
            return 0
                
    else:
        assert isinstance(target_output, type(expected_output))
        return 1
            
def equation_output_check(expected_output, target_output, error):
    success = 0
    if isinstance(target_output, dict):
        for key in target_output.keys():
            try:
                success += equation_output_check(expected_output[key], 
                                         target_output[key], error)
            except:
                print(expected_output[key], 
                                         target_output[key])
                print("Error: {} for variable {}.".format(error,
                                                                          key))
        if success == len(target_output.keys()):
            return 1
        else:
            return 0
    elif isinstance(target_output, tuple) or isinstance(target_output, list):
        for i in range(len(target_output)):
            try: 
                success += equation_output_check(expected_output[i], 
                                         target_output[i], error)
            except:
                print("Error: {} for variable in position {}.".format(error, i))
        if success == len(target_output):
            return 1
        else:
            return 0
                
    else:
        if hasattr(target_output, 'shape'):
            np.testing.assert_array_almost_equal(target_output, expected_output)
        else:
            assert target_output == expected_output
        return 1
    
def shape_check(expected_output, target_output, error):
    success = 0
    if isinstance(target_output, dict):
        for key in target_output.keys():
            try:
                success += shape_check(expected_output[key], 
                                         target_output[key], error)
            except:
                print("Error: {} for variable {}.".format(error, key))
        if success == len(target_output.keys()):
            return 1
        else:
            return 0
    elif isinstance(target_output, tuple) or isinstance(target_output, list):
        for i in range(len(target_output)):
            try: 
                success += shape_check(expected_output[i], 
                                         target_output[i], error)
            except:
                print("Error: {} for variable {}.".format(error, i))
        if success == len(target_output):
            return 1
        else:
            return 0
                
    else:
        if hasattr(target_output, 'shape'):
            assert target_output.shape == expected_output.shape
        return 1
                
def multiple_test(test_cases, target):
    success = 0
    for test_case in test_cases:
        try:
            target_answer = target(*test_case['input'])
            if test_case['name'] == "datatype_check":
                success += datatype_check(test_case['expected'], target_answer, test_case['error'])
            if test_case['name'] == "equation_output_check":
                success += equation_output_check(test_case['expected'], target_answer, test_case['error'])
            if test_case['name'] == "shape_check":
                success += shape_check(test_case['expected'], target_answer, test_case['error'])
        except:
            print("Error: " + test_case['error'])
            
    if success == len(test_cases):
        print("\033[92m All tests passed.")
    else:
        print('\033[92m', success," Tests passed")
        print('\033[91m', len(test_cases) - success, " Tests failed")
        raise AssertionError("Not all tests were passed for {}. Check your equations and avoid using global variables inside the function.".format(target.__name__))


### general tests

np.random.seed(3)

### ex1
def frac_censored_test(target, data):
    data = data
    print("Observations which were censored: ", target(data))
    expected_output = 0.325
    
    test_cases = [
        {
            "name":"datatype_check",
            "input": [data],
            "expected": expected_output,
            "error": "Data-type mismatch."
        },
        {
            "name": "shape_check",
            "input": [data],
            "expected": expected_output,
            "error": "Wrong shape."
        },
        {
            "name": "equation_output_check",
            "input": [data],
            "expected": expected_output,
            "error": "Wrong output."
        }
    ]
    
    multiple_test(test_cases, target)
    

##############################################        
### ex2
def naive_estimator_test_case():
    sample_df_1 = pd.DataFrame(columns = ["Time", "Event"])
    sample_df_1.Time = [5, 10, 15]
    sample_df_1.Event = [0, 1, 0]
    
    sample_df_2 = pd.DataFrame({'Time': [5,5,10],
                                'Event': [0,1,0]
                               })
    
    return sample_df_1, sample_df_2

def naive_estimator_test(target):
    sample_df_1, sample_df_2 = naive_estimator_test_case()
    
    print("Sample 1 dataframe for testing code:\n")
    print(sample_df_1)
    print("\n")
    
    print("Test Case 1: S(3)")
    print("Output: ", target(3, sample_df_1))

    print("\nTest Case 2: S(12)")
    print("Output: ", target(12, sample_df_1))

    print("\nTest Case 3: S(20)")
    print("Output: ", target(20, sample_df_1))
    
    print("\nSample 2 dataframe for testing code:\n")
    print("\n", sample_df_2, "\n")

    print("Test case 4: S(5)")
    print("Output: ", target(5, sample_df_2), "\n")
    
    expected_output_1 = 1.0
    expected_output_2 = 0.5
    expected_output_3 = 0.0
    expected_output_4 = 0.5
    
    test_cases = [
        {
            "name":"datatype_check",
            "input": [3, sample_df_1],
            "expected": expected_output_1,
            "error": "Data-type mismatch for Test Case 1."
        },
        {
            "name": "shape_check",
            "input": [3, sample_df_1],
            "expected": expected_output_1,
            "error": "Wrong shape for Test Case 1."
        },
        {
            "name": "equation_output_check",
            "input": [3, sample_df_1],
            "expected": expected_output_1,
            "error": "Wrong output for Test Case 1."
        },
        {
            "name": "equation_output_check",
            "input": [12, sample_df_1],
            "expected": expected_output_2,
            "error": "Wrong output for Test Case 2."
        },
        {
            "name": "equation_output_check",
            "input": [20, sample_df_1],
            "expected": expected_output_3,
            "error": "Wrong output for Test Case 3."
        },
        {
            "name": "equation_output_check",
            "input": [5, sample_df_2],
            "expected": expected_output_4,
            "error": "Wrong output for Test Case 4."
        }
    ]
    
    multiple_test(test_cases, target)
    


##############################################        
### ex3
def HomemadeKM_test_case():
    sample_df_1 = pd.DataFrame(columns = ["Time", "Event"])
    sample_df_1.Time = [5, 10, 15]
    sample_df_1.Event = [0, 1, 0]
    
    sample_df_2 = pd.DataFrame(columns = ["Time", "Event"])
    sample_df_2.loc[:, "Time"] = [2, 15, 12, 10, 20]
    sample_df_2.loc[:, "Event"] = [0, 0, 1, 1, 1]
    
    return sample_df_1, sample_df_2

def HomemadeKM_test(target):
    
    sample_df_1, sample_df_2 = HomemadeKM_test_case()
    
    print("Test Case 1\n")
    print(sample_df_1.head(), "\n")
    x, y = target(sample_df_1)
    print("Test Case 1 Event times: {}, Survival Probabilities: {}".format(x, y))
    
    print("\nTest Case 2\n")
    print(sample_df_2.head(), "\n")
    x, y = target(sample_df_2)
    print("Test Case 2 Event times: {}, Survival Probabilities: {}".format(x, y), "\n")
    
    expected_output_1 = (np.array([0, 5, 10, 15]), np.array([1.0, 1.0, 0.5, 0.5]))
    expected_output_2 = (np.array([0, 2, 10, 12, 15, 20]), np.array([1.0, 1.0, 0.75, 0.5, 0.5, 0.0]))
    
    test_cases = [
        
        {
            "name": "shape_check",
            "input": [sample_df_1],
            "expected": expected_output_1,
            "error": "Wrong shape for Test Case 1."
        },
        {
            "name": "equation_output_check",
            "input": [sample_df_1],
            "expected": expected_output_1,
            "error": "Wrong output for Test Case 1."
        },
        {
            "name": "equation_output_check",
            "input": [sample_df_2],
            "expected": expected_output_2,
            "error": "Wrong output for Test Case 2."
        }
    ]
    
    multiple_test(test_cases, target)
    
    
    

<a name='2'></a>
## 2. Load the Dataset


Run the next cell to load the lymphoma data set. 

In [ ]:
def load_data():
    df = load_lymphoma()
    df.loc[:, 'Event'] = df.Censor
    df = df.drop(['Censor'], axis=1)
    return df

data = load_data()

As always, you first look over your data.

In [ ]:
print("data shape: {}".format(data.shape))
data.head()

The column `Time` states how long the patient lived before they died or were censored.

The column `Event` says whether a death was observed or not. `Event` is 1 if the event is observed (i.e. the patient died) and 0 if data was censored.

Censorship here means that the observation has ended without any observed event.
For example, let a patient be in a hospital for 100 days at most. If a patient dies after only 44 days, their event will be recorded as `Time` = 44 and `Event` = 1. If a patient walks out after 100 days and dies 3 days later (103 days total), this event is not observed in our process and the corresponding row has `Time` = 100 and `Event` = 0. If a patient survives for 25 years after being admitted, their data for are still `Time` = 100 and `Event` = 0.

<a name='3'></a>
## 3. Censored Data

We can plot a histogram of the survival times to see in general how long cases survived before censorship or events.

In [ ]:
data.Time.hist();
plt.xlabel("Observation time before death or censorship (days)");
plt.ylabel("Frequency (number of patients)");
# Note that the semicolon at the end of the plotting line
# silences unnecessary textual output - try removing it
# to observe its effect

<a name='ex-1'></a>
### Exercise 1 - frac_censored

In the next cell, write a function to compute the fraction ($\in [0, 1]$) of observations which were censored. 

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<p>
<ul>
    <li>Summing up the <code>'Event'</code> column will give you the number of observations where censorship has NOT occurred.</li>
    
</ul>
</p>

In [ ]:

def frac_censored(df):
    """
    Return percent of observations which were censored.
    
    Args:
        df (dataframe): dataframe which contains column 'Event' which is 
                        1 if an event occurred (death)
                        0 if the event did not occur (censored)
    Returns:
        frac_censored (float): fraction of cases which were censored. 
    """
    result = 0.0
    
    censored_count = sum(df['Event'] == 0)
    result = censored_count/len(df)
    
    return result

In [ ]:
frac_censored_test(frac_censored, data)

#### Expected Output:
```
Observations which were censored:  0.325
 All tests passed.
```

Run the next cell to see the distributions of survival times for censored and uncensored examples.

In [ ]:
df_censored = data[data.Event == 0]
df_uncensored = data[data.Event == 1]

df_censored.Time.hist()
plt.title("Censored")
plt.xlabel("Time (days)")
plt.ylabel("Frequency")
plt.show()

df_uncensored.Time.hist()
plt.title("Uncensored")
plt.xlabel("Time (days)")
plt.ylabel("Frequency")
plt.show()

<a name='4'></a>
## 4. Survival Estimates

We'll now try to estimate the survival function:

$$
S(t) = P(T > t)
$$

To illustrate the strengths of Kaplan Meier, we'll start with a naive estimator of the above survival function. To estimate this quantity, we'll divide the number of people who we know lived past time $t$ by the number of people who were not censored before $t$.

Formally, let $i$ = 1, ..., $n$ be the cases, and let $T_i$ be the time when $i$ was censored or an event happened. Let $e_i= 1$ if an event was observed for $i$ and 0 otherwise. Then let $X_t = \{i : T_i > t\}$, and let $M_t = \{i : e_i = 1 \text{ or } T_i > t\}$. The estimator you will compute will be:

$$
\hat{S}(t) = \frac{|X_t|}{|M_t|}
$$



<a name='ex-2'></a>
### Exercise 2 - naive_estimator
Write a function to compute this estimate for arbitrary $t$ in the cell below.

In [ ]:
def naive_estimator(t, df):
    """
    Return naive estimate for S(t), the probability
    of surviving past time t. Given by number
    of cases who survived past time t divided by the
    number of cases who weren't censored before time t.
    
    Args:
        t (int): query time
        df (dataframe): survival data. Has a Time column,
                        which says how long until that case
                        experienced an event or was censored,
                        and an Event column, which is 1 if an event
                        was observed and 0 otherwise.
    Returns:
        S_t (float): estimator for survival function evaluated at t.
    """
    S_t = 0.0
    X = sum(df['Time'] > t)
    M = sum( (df['Time'] > t) | (df['Event'] == 1) )
    S_t = X / M
    
    return S_t

In [ ]:

naive_estimator_test(naive_estimator)    

#### Expected Output:
```
Test Case 1: S(3)
Output:  1.0

Test Case 2: S(12)
Output:  0.5

Test Case 3: S(20)
Output:  0.0

Test case 4: S(5)
Output:  0.5 

 All tests passed.

```

In the next cell, we will plot the naive estimator using the real data up to the maximum time in the dataset. 

In [ ]:
max_time = data.Time.max()
x = range(0, max_time+1)
y = np.zeros(len(x))
for i, t in enumerate(x):
    y[i] = naive_estimator(t, data)
    
plt.plot(x, y)
plt.title("Naive Survival Estimate")
plt.xlabel("Time")
plt.ylabel("Estimated cumulative survival rate")
plt.show()

<a name='ex-3'></a>
### Exercise 3 - HomemadeKM

Next let's compare this with the Kaplan Meier estimate. In the cell below, write a function that computes the Kaplan Meier estimate of $S(t)$ at every distinct time in the dataset. 

Recall the Kaplan-Meier estimate:

$$
S(t) = \prod_{t_i \leq t} (1 - \frac{d_i}{n_i})
$$

where $t_i$ are the events observed in the dataset and $d_i$ is the number of deaths at time $t_i$ and $n_i$ is the number of people who we know have survived up to time $t_i$.

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<p>
<ul>
    <li>Try sorting by Time.</li>
    <li>Use <a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.unique.html">pandas.Series.unique<a> </li>
    <li>If you get a division by zero error, please double-check how you calculated `n_t`</li>
</ul>
</p>

In [ ]:

def HomemadeKM(df):
    """
    Return KM estimate evaluated at every distinct
    time (event or censored) recorded in the dataset.
    Event times and probabilities should begin with
    time 0 and probability 1.
    
    Example:
    
    input: 
    
         Time  Censor
    0     5       0
    1    10       1
    2    15       0
    
    correct output: 
    
    event_times: [0, 5, 10, 15]
    S: [1.0, 1.0, 0.5, 0.5]
    
    Args:
        df (dataframe): dataframe which has columns for Time
                          and Event, defined as usual.
                          
    Returns:
        event_times (list of ints): array of unique event times
                                      (begins with 0).
        S (list of floats): array of survival probabilites, so that
                            S[i] = P(T > event_times[i]). This 
                            begins with 1.0 (since no one dies at time
                            0).
    """
    # individuals are considered to have survival probability 1
    # at time 0
    event_times = [0]
    p = 1.0
    S = [p]
    
    # get collection of unique observed event times
    observed_event_times = df.Time.unique()
  
    # sort event times
    observed_event_times = sorted(observed_event_times)
    
    # iterate through event times
    for t in observed_event_times:
  
        # compute n_t, number of people who survive to time t
        n_t = len(df[df.Time >= t])
  
        # compute d_t, number of people who die at time t
        d_t = len(df[(df.Time == t) & (df.Event == 1)])
        
        # update p
        p = p*(1 - float(d_t)/n_t)
  
        # update S and event_times (ADD code below)
        # hint: use append
        event_times.append(t)
        S.append(p)
  
    return event_times, S

In [ ]:
HomemadeKM_test(HomemadeKM)    

#### Expected Output:
```
Test Case 1 Event times: [0, 5, 10, 15], Survival Probabilities: [1.0, 1.0, 0.5, 0.5]
Test Case 2 Event times: [0, 2, 10, 12, 15, 20], Survival Probabilities: [1.0, 1.0, 0.75, 0.5, 0.5, 0.0] 

 All tests passed.
```

Now let's plot the two against each other on the data to see the difference.

In [ ]:
max_time = data.Time.max()
x = range(0, max_time+1)
y = np.zeros(len(x))
for i, t in enumerate(x):
    y[i] = naive_estimator(t, data)
    
plt.plot(x, y, label="Naive")

x, y = HomemadeKM(data)
plt.step(x, y, label="Kaplan-Meier")
plt.xlabel("Time")
plt.ylabel("Survival probability estimate")
plt.legend()
plt.show()

### Question

What differences do you observe between the naive estimator and Kaplan-Meier estimator? Do any of our earlier explorations of the dataset help to explain these differences?

<a name='5'></a>
## 5. Subgroup Analysis

We see that along with Time and Censor, we have a column called `Stage_group`. 
- A value of 1 in this column denotes a patient with stage III cancer
- A value of 2 denotes stage IV. 

We want to compare the survival functions of these two groups.

This time we'll use the `KaplanMeierFitter` class from `lifelines`. Run the next cell to fit and plot the Kaplan Meier curves for each group. 

In [ ]:
S1 = data[data.Stage_group == 1]
km1 = KM()
km1.fit(S1.loc[:, 'Time'], event_observed = S1.loc[:, 'Event'], label = 'Stage III')

S2 = data[data.Stage_group == 2]
km2 = KM()
km2.fit(S2.loc[:, "Time"], event_observed = S2.loc[:, 'Event'], label = 'Stage IV')

ax = km1.plot(ci_show=False)
km2.plot(ax = ax, ci_show=False)
plt.xlabel('time')
plt.ylabel('Survival probability estimate')
plt.savefig('two_km_curves', dpi=300)

Let's compare the survival functions at 90, 180, 270, and 360 days

In [ ]:
survivals = pd.DataFrame([90, 180, 270, 360], columns = ['time'])
survivals.loc[:, 'Group 1'] = km1.survival_function_at_times(survivals['time']).values
survivals.loc[:, 'Group 2'] = km2.survival_function_at_times(survivals['time']).values

In [ ]:
survivals

This makes clear the difference in survival between the Stage III and IV cancer groups in the dataset. 

<a name='5-1'></a>
## 5.1 Bonus: Log-Rank Test

To say whether there is a statistical difference between the survival curves we can run the log-rank test. This test tells us the probability that we could observe this data if the two curves were the same. The derivation of the log-rank test is somewhat complicated, but luckily `lifelines` has a simple function to compute it. 

Run the next cell to compute a p-value using `lifelines.statistics.logrank_test`.

In [ ]:
def logrank_p_value(group_1_data, group_2_data):
    result = logrank_test(group_1_data.Time, group_2_data.Time,
                          group_1_data.Event, group_2_data.Event)
    return result.p_value

logrank_p_value(S1, S2)

0.009588929834755544

If everything is correct, you should see a p value of less than 0.05, which indicates that the difference in the curves is indeed statistically significant.

# **C.** Cox Proportional Hazards (linear) and Random Survival Forests (non-linear)

Welcome to the final assignment in Course 2! In this assignment you'll develop risk models using survival data and a combination of linear and non-linear techniques. We'll be using a dataset with survival data of patients with Primary Biliary Cirrhosis (pbc). PBC is a progressive disease of the liver caused by a buildup of bile within the liver (cholestasis) that results in damage to the small bile ducts that drain bile from the liver. Our goal will be to understand the effects of different factors on the survival times of the patients. Along the way you'll learn about the following topics: 

- Cox Proportional Hazards
  - Data Preprocessing for Cox Models.
- Random Survival Forests
  - Permutation Methods for Interpretation.

## Table of Contents

- [1. Import Packages](#1)
- [2. Load the Dataset](#2)
- [3. Explore the Dataset](#3)
- [4. Cox Proportional Hazards](#4)
    - [Exercise 1 - to_one_hot](#ex-1)
- [5. Fitting and Interpreting a Cox Model](#5)
- [6. Hazard ratio](#3)
    - [Exercise 2 - hazard_ratio](#ex-2)
- [7. Harrell's C-Index](#7)
    - [Exercise 3 - harrell_c](#ex-3)
- [8. Random Survival Forests](#8)
- [9. Permutation Method for Interpretation](#9)

<a name='1'></a>
## 1. Import Packages

We'll first import all the packages that we need for this assignment. 

- `sklearn` is one of the most popular machine learning libraries.
- `numpy` is the fundamental package for scientific computing in python.
- `pandas` is what we'll use to manipulate our data.
- `matplotlib` is a plotting library.
- `lifelines` is an open-source survival analysis library.

In [ ]:
!pip install -q lifelines

In [ ]:
import os
import sys
from IPython.display import display

import sklearn
from sklearn.model_selection import train_test_split
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from lifelines import CoxPHFitter
from lifelines.utils import concordance_index as cindex

In [ ]:
#@title tests

### tests for data

np.random.seed(3)

def datatype_check(expected_output, target_output, error):
    success = 0
    if isinstance(target_output, dict):
        for key in target_output.keys():
            try:
                success += datatype_check(expected_output[key], 
                                         target_output[key], error)
            except:
                print("Error: {} in variable {}. Got {} but expected type {}".format(error,
                                                                          key, type(target_output[key]), type(expected_output[key])))
        if success == len(target_output.keys()):
            return 1
        else:
            return 0
    elif isinstance(target_output, tuple) or isinstance(target_output, list):
        for i in range(len(target_output)):
            try: 
                success += datatype_check(expected_output[i], 
                                         target_output[i], error)
            except:
                print("Error: {} in variable {}, Got {}  but expected type {}".format(error,
                                                                          i, type(target_output[i]), type(expected_output[i])))
        if success == len(target_output):
            return 1
        else:
            return 0
                
    else:
        assert isinstance(target_output, type(expected_output))
        return 1
            
def equation_output_check(expected_output, target_output, error):
    success = 0
    if isinstance(target_output, dict):
        for key in target_output.keys():
            try:
                success += equation_output_check(expected_output[key], 
                                         target_output[key], error)
            except:
                print(expected_output[key], 
                                         target_output[key])
                print("Error: {} for variable {}.".format(error,
                                                                          key))
        if success == len(target_output.keys()):
            return 1
        else:
            return 0
    elif isinstance(target_output, tuple) or isinstance(target_output, list):
        for i in range(len(target_output)):
            try: 
                success += equation_output_check(expected_output[i], 
                                         target_output[i], error)
            except:
                print("Error: {} for variable in position {}.".format(error, i))
        if success == len(target_output):
            return 1
        else:
            return 0
                
    else:
        if hasattr(target_output, 'shape'):
            np.testing.assert_array_almost_equal(target_output, expected_output)
        else:
            assert target_output == expected_output
        return 1
    
def shape_check(expected_output, target_output, error):
    success = 0
    if isinstance(target_output, dict):
        for key in target_output.keys():
            try:
                success += shape_check(expected_output[key], 
                                         target_output[key], error)
            except:
                print("Error: {} for variable {}.".format(error, key))
        if success == len(target_output.keys()):
            return 1
        else:
            return 0
    elif isinstance(target_output, tuple) or isinstance(target_output, list):
        for i in range(len(target_output)):
            try: 
                success += shape_check(expected_output[i], 
                                         target_output[i], error)
            except:
                print("Error: {} for variable {}.".format(error, i))
        if success == len(target_output):
            return 1
        else:
            return 0
                
    else:
        if hasattr(target_output, 'shape'):
            assert target_output.shape == expected_output.shape
        return 1
                
def multiple_test(test_cases, target):
    success = 0
    for test_case in test_cases:
        try:
            target_answer = target(*test_case['input'])
            if test_case['name'] == "datatype_check":
                success += datatype_check(test_case['expected'], target_answer, test_case['error'])
            if test_case['name'] == "equation_output_check":
                success += equation_output_check(test_case['expected'], target_answer, test_case['error'])
            if test_case['name'] == "shape_check":
                success += shape_check(test_case['expected'], target_answer, test_case['error'])
        except:
            print("Error: " + test_case['error'])
            
    if success == len(test_cases):
        print("\033[92m All tests passed.")
    else:
        print('\033[92m', success," Tests passed")
        print('\033[91m', len(test_cases) - success, " Tests failed")
        raise AssertionError("Not all tests were passed for {}. Check your equations and avoid using global variables inside the function.".format(target.__name__))

### general tests

np.random.seed(3)

### ex1
def to_one_hot_test(target_to_one_hot, df_train, df_val, df_test):
    to_encode = ['spiders', 'stage']
    
    def to_list(target_to_one_hot, to_encode, df_train, df_val, df_test):
        one_hot_train = target_to_one_hot(df_train, to_encode)
        one_hot_val = target_to_one_hot(df_val, to_encode)
        one_hot_test = target_to_one_hot(df_test, to_encode)
        
        return one_hot_train.columns.tolist(), one_hot_val.columns.tolist(), one_hot_test.columns.tolist()
    
    one_hot_train_to_list, one_hot_val_to_list, one_hot_test_to_list = to_list(target_to_one_hot, to_encode, df_train, df_val, df_test)
    
    print("One hot val columns:\n\n", one_hot_val_to_list, "\n")
    print("There are", len(one_hot_val_to_list), "columns\n")
    
    expected_output = (['time', 'status', 'trt', 'age', 'sex', 'ascites', 'hepato', 'edema', 'bili', 
                        'chol', 'albumin', 'copper', 'alk.phos', 'ast', 'trig', 'platelet', 'protime', 'spiders_1.0',
                        'stage_2.0', 'stage_3.0', 'stage_4.0'], 
                       ['time', 'status', 'trt', 'age', 'sex', 'ascites', 'hepato', 'edema', 'bili', 'chol', 'albumin', 
                        'copper', 'alk.phos', 'ast', 'trig', 'platelet', 'protime', 'spiders_1.0', 'stage_2.0', 'stage_3.0',
                        'stage_4.0'], 
                       ['time', 'status', 'trt', 'age', 'sex', 'ascites', 'hepato', 'edema', 'bili', 'chol', 'albumin', 'copper',
                        'alk.phos', 'ast', 'trig', 'platelet', 'protime', 'spiders_1.0', 'stage_2.0', 'stage_3.0', 'stage_4.0'])
        
        
    test_cases = [
        {
            "name":"datatype_check",
            "input": [target_to_one_hot, to_encode, df_train, df_val, df_test],
            "expected": expected_output,
            "error": "Data-type mismatch."
        },
        {
            "name": "shape_check",
            "input": [target_to_one_hot, to_encode, df_train, df_val, df_test],
            "expected": expected_output,
            "error": "Wrong shape."
        },
        {
            "name": "equation_output_check",
            "input": [target_to_one_hot, to_encode, df_train, df_val, df_test],
            "expected": expected_output,
            "error": "Wrong output."
        }
    ]
    
    multiple_test(test_cases, to_list)
    


##############################################        
### ex2
def hazard_ratio_test_case(i, j, one_hot_train):
    case_1 = one_hot_train.iloc[i, :].drop(['time', 'status'])
    case_2 = one_hot_train.iloc[j, :].drop(['time', 'status'])
    
    return case_1, case_2


def hazard_ratio_test(target, i, j, one_hot_train, cph):
    case_1, case_2 = hazard_ratio_test_case(i, j, one_hot_train)
    
    print(target(case_1.values, case_2.values, cph.params_.values), "\n")
    
    expected_output = np.float64(15.029022704011822)  #np.float64(15.029017732492221)
    
    test_cases = [
        {
            "name":"datatype_check",
            "input": [case_1.values, case_2.values, cph.params_.values],
            "expected": expected_output,
            "error": "Data-type mismatch."
        },
        {
            "name": "shape_check",
            "input": [case_1.values, case_2.values, cph.params_.values],
            "expected": expected_output,
            "error": "Wrong shape."
        },
        {
            "name": "equation_output_check",
            "input": [case_1.values, case_2.values, cph.params_.values],
            "expected": expected_output,
            "error": "Wrong output. One possible solution: make sure i = 1 and j = 5"
        }
    ]
    
    multiple_test(test_cases, target)
    


##############################################        
### ex3
def harrell_c_test_case():
    y_true_1 = [30, 12, 84, 9]
    
    event_1 = [1, 1, 1, 1]
    scores_1 = [0.5, 0.9, 0.1, 1.0]
    
    scores_2 = [0.9, 0.5, 1.0, 0.1]
    
    event_3 = [1, 0, 1, 1]
    scores_3 = [0.5, 0.9, 0.1, 1.0]
    
    y_true_4 = [30, 30, 20, 20]
    event_4 = [1, 0, 1, 0]
    scores_4 = [10, 5, 15, 20]
    
    y_true_5 = list(reversed([30, 30, 30, 20, 20]))
    event_5 = [0, 1, 0, 1, 0]
    scores_5 = list(reversed([15, 10, 5, 15, 20]))
    
    y_true_6 = [10,10]
    event_6 = [0,1]
    scores_6 = [4,5]
    
    return y_true_1, event_1, scores_1, scores_2, event_3, scores_3, y_true_4, event_4, scores_4, y_true_5, event_5, scores_5, y_true_6, event_6, scores_6


def harrell_c_test(target):
    
    y_true_1, event_1, scores_1, scores_2, event_3, scores_3, y_true_4, event_4, scores_4, y_true_5, event_5, scores_5, y_true_6, event_6, scores_6 = harrell_c_test_case()
    
    print("Test Case 1\n")
    print("y_true: ", y_true_1)
    print("scores: ", scores_1)
    print("event:  ", event_1)
    print("Output: ", target(y_true_1, scores_1, event_1))
    expected_output_1 = 1.0
    
    print("\nTest Case 2\n")
    print("y_true: ", y_true_1)
    print("scores: ", scores_2)
    print("event:  ", event_1)
    print("Output: ", target(y_true_1, scores_2, event_1))
    expected_output_2 = 0.0
    
    print("\nTest Case 3\n")
    print("y_true: ", y_true_1)
    print("scores: ", scores_3)
    print("event:  ", event_3)
    print("Output: ", target(y_true_1, scores_3, event_3))
    expected_output_3 = 1.0
    
    print("\nTest Case 4\n")
    print("y_true: ", y_true_4)
    print("scores: ", scores_4)
    print("event:  ", event_4)
    print("Output: ", target(y_true_4, scores_4, event_4))
    expected_output_4 = 0.75
    
    print("\nTest Case 5\n")
    print("y_true: ", y_true_5)
    print("scores: ", scores_5)
    print("event:  ", event_5)
    print("Output: ", target(y_true_5, scores_5, event_5))
    expected_output_5 = 0.5833333333333334
    
    print("\nTes Case 6\n")
    print("y_true: ", y_true_6)
    print("scores: ", scores_6)
    print("event:  ", event_6)
    print("Output: ", target(y_true_6, scores_6, event_6), "\n")
    expected_output_6 = 1.0
    
    test_cases = [
        {
            "name":"datatype_check",
            "input": [y_true_1, scores_1, event_1],
            "expected": expected_output_1,
            "error": "Data-type mismatch for Test Case 1."
        },
        {
            "name": "shape_check",
            "input": [y_true_1, scores_1, event_1],
            "expected": expected_output_1,
            "error": "Wrong shape for Test Case 1."
        },
        {
            "name": "equation_output_check",
            "input": [y_true_1, scores_1, event_1],
            "expected": expected_output_1,
            "error": "Wrong output for Test Case 1."
        },
        {
            "name": "equation_output_check",
            "input": [y_true_1, scores_2, event_1],
            "expected": expected_output_2,
            "error": "Wrong output for Test Case 2."
        },
        {
            "name": "equation_output_check",
            "input": [y_true_1, scores_3, event_3],
            "expected": expected_output_3,
            "error": "Wrong output for Test Case 3."
        },
        {
            "name": "equation_output_check",
            "input": [y_true_4, scores_4, event_4],
            "expected": expected_output_4,
            "error": "Wrong output for Test Case 4."
        },
        {
            "name": "equation_output_check",
            "input": [y_true_5, scores_5, event_5],
            "expected": expected_output_5,
            "error": "Wrong output for Test Case 5."
        },
        {
            "name": "equation_output_check",
            "input": [y_true_6, scores_6, event_6],
            "expected": expected_output_6,
            "error": "Wrong output for Test Case 6."
        }
    ]
    
    multiple_test(test_cases, target)
        


<a name='2'></a>
## 2. Load the Dataset

Run the next cell to load the data.

In [ ]:
def load_data(data):
    df = pd.read_csv(data)
    df = df.drop('id', axis=1)
    df = df[df.status != 1]
    df.loc[:, 'status'] = df.status / 2.0
    df.loc[:, 'time'] = df.time / 365.0
    df.loc[:, 'trt'] = df.trt - 1
    df.loc[:, 'sex'] = df.sex.map({'f':0.0, 'm':1.0})
    df = df.dropna(axis=0)

    return df

df = load_data('./pbc.csv')

<a name='3'></a>
## 3. Explore the Dataset

In the lecture videos `time` was in months, however in this assignment, `time` will be converted into years. Also notice that we have assigned a numeric value to `sex`, where `female = 0` and `male = 1`.

Next, familiarize yourself with the data and the shape of it. 

In [ ]:
print(df.shape)

# df.head() only outputs the top few rows
df.head()

Take a minute to examine particular cases.

In [ ]:
i = 20
df.iloc[i, :]

Now, split your dataset into train, validation and test set using 60/20/20 split. 

In [ ]:
np.random.seed(0)
df_dev, df_test = train_test_split(df, test_size = 0.2)
df_train, df_val = train_test_split(df_dev, test_size = 0.25)

print("Total number of patients:", df.shape[0])
print("Total number of patients in training set:", df_train.shape[0])
print("Total number of patients in validation set:", df_val.shape[0])
print("Total number of patients in test set:", df_test.shape[0])

Before proceeding to modeling, let's normalize the continuous covariates to make sure they're on the same scale. Again, we should normalize the test data using statistics from the train data.

In [ ]:
continuous_columns = ['age', 'bili', 'chol', 'albumin', 'copper', 'alk.phos', 'ast', 'trig', 'platelet', 'protime']
mean = df_train.loc[:, continuous_columns].mean()
std = df_train.loc[:, continuous_columns].std()
df_train.loc[:, continuous_columns] = (df_train.loc[:, continuous_columns] - mean) / std
df_val.loc[:, continuous_columns] = (df_val.loc[:, continuous_columns] - mean) / std
df_test.loc[:, continuous_columns] = (df_test.loc[:, continuous_columns] - mean) / std

Let's check the summary statistics on our training dataset to make sure it's standardized.

In [ ]:
df_train.loc[:, continuous_columns].describe()

<a name='4'></a>
## 4. Cox Proportional Hazards

Our goal is to build a risk score using the survival data that we have. We'll begin by fitting a Cox Proportional Hazards model to your data.

Recall that the Cox Proportional Hazards model describes the hazard for an individual $i$ at time $t$ as 

$$
\lambda(t, x) = \lambda_0(t)e^{\theta^T X_i}
$$

The $\lambda_0$ term is a baseline hazard and incorporates the risk over time, and the other term incorporates the risk due to the individual's covariates. After fitting the model, we can rank individuals using the person-dependent risk term $e^{\theta^T X_i}$. 

Categorical variables cannot be used in a regression model as they are. In order to use them, conversion to a series of variables is required.

Since our data has a mix of categorical (`stage`) and continuous (`wblc`) variables, before we proceed further we need to do some data engineering. To tackle the issue at hand we'll be using the `Dummy Coding` technique. In order to use Cox Proportional Hazards, we will have to turn the categorical data into one hot features so that we can fit our Cox model. Luckily, Pandas has a built-in function called `get_dummies` that will make it easier for us to implement our function. It turns categorical features into multiple binary features.





<a name='ex-1'></a>
### Exercise 1 - to_one_hot
In the cell below, implement the `to_one_hot(...)` function.

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<p>
<ul>
    <li>Remember to drop the first dummy for each each category to avoid convergence issues when fitting the proportional hazards model.</li>
    <li> Check out the <a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html" > get_dummies() </a>  documentation. </li>
    <li>Use <code>dtype=np.float64</code>.</li>
</ul>
</p>

In [ ]:

def to_one_hot(dataframe, columns):
    '''
    Convert columns in dataframe to one-hot encoding.
    Args:
        dataframe (dataframe): pandas dataframe containing covariates
        columns (list of strings): list categorical column names to one hot encode
    Returns:
        one_hot_df (dataframe): dataframe with categorical columns encoded
                            as binary variables
    '''
    
    one_hot_df = pd.get_dummies(dataframe, columns = columns, drop_first = True, dtype=np.float64)
    
    return one_hot_df

In [ ]:
# test cell ex1 - do not modify this test cell
to_one_hot_test(to_one_hot, df_train, df_val, df_test)

#### Expected Output:
```
One hot val columns:

 ['time', 'status', 'trt', 'age', 'sex', 'ascites', 'hepato', 'edema', 'bili', 'chol', 'albumin', 'copper', 'alk.phos', 'ast', 'trig', 'platelet', 'protime', 'spiders_1.0', 'stage_2.0', 'stage_3.0', 'stage_4.0'] 

There are 21 columns

 All tests passed.
```

Now you'll use the function you coded to transform the training, validation, and test sets.

In [ ]:
to_encode = ['edema', 'stage']

one_hot_train = to_one_hot(df_train, to_encode)
one_hot_val = to_one_hot(df_val, to_encode)
one_hot_test = to_one_hot(df_test, to_encode)

### Look for new features
Now, let's take a peek at one of the transformed data sets. Do you notice any new features?

In [ ]:
print(one_hot_train.shape)
one_hot_train.head()

<a name='5'></a>
## 5. Fitting and Interpreting a Cox Model

Run the following cell to fit your Cox Proportional Hazards model using the `lifelines` package.

In [ ]:
cph = CoxPHFitter()
cph.fit(one_hot_train, duration_col = 'time', event_col = 'status', fit_options = {'step_size':0.1})

<lifelines.CoxPHFitter: fitted with 154 total observations, 90 right-censored observations>

You can use `cph.print_summary()` to view the coefficients associated with each covariate as well as confidence intervals. 

In [ ]:
cph.print_summary()

**Question:**

- According to the model, was treatment `trt` beneficial? 
- What was its associated hazard ratio? 
    - Note that the hazard ratio is how much an incremental increase in the feature variable changes the hazard.

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Check your answer!</b></font>
</summary>
<p>
<ul>
<ul>
    <li>You should see that the treatment (trt) was beneficial because it has a negative impact on the hazard (the coefficient is negative, and exp(coef) is less than 1).</li>
    <li>The associated hazard ratio is ~0.8, because this is the exp(coef) of treatment.</li>
</ul>
</p>

We can compare the predicted survival curves for treatment variables. Run the next cell to plot survival curves using the `plot_covariate_groups()` function. 
- The y-axis is th survival rate
- The x-axis is time

In [ ]:
cph.plot_covariate_groups('trt', values=[0, 1]);

Notice how the group without treatment has a lower survival rate at all times (the x-axis is time) compared to the treatment group.

<a name='6'></a>
## 6. Hazard Ratio

Recall from the lecture videos that the Hazard Ratio between two patients was the likelihood of one patient (e.g smoker) being more at risk than the other (e.g non-smoker).
$$
\frac{\lambda_{smoker}(t)}{\lambda_{nonsmoker}(t)} = e^{\theta (X_{smoker} - X_{nonsmoker})^T}
$$

Where

$$
\lambda_{smoker}(t) = \lambda_0(t)e^{\theta X_{smoker}^T}
$$
and
$$
\lambda_{nonsmoker}(t) = \lambda_0(t)e^{\theta X_{nonsmoker}^T} \\
$$

<a name='ex-2'></a>
### Exercise 2 - hazard_ratio
In the cell below, write a function to compute the hazard ratio between two individuals given the cox model's coefficients.

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<p>
<ul>
    <li>use numpy.dot</li>
    <li>use nump.exp</li>
</ul>
</p>


In [ ]:
def hazard_ratio(case_1, case_2, cox_params):
    '''
    Return the hazard ratio of case_1 : case_2 using
    the coefficients of the cox model.
    
    Args:
        case_1 (np.array): (1 x d) array of covariates
        case_2 (np.array): (1 x d) array of covariates
        model (np.array): (1 x d) array of cox model coefficients
    Returns:
        hazard_ratio (float): hazard ratio of case_1 : case_2
    '''
    
    hr = np.exp(cox_params.dot((case_1 - case_2).T))
    
    return hr

Now, evaluate it on the following pair of indivduals: `i = 1` and `j = 5`. Given your implementation of `hazard_ratio` is correct, you will **only** pass the test when `i = 1` and `j = 5`.

In [ ]:
# test cell ex2

# Set i = 1 and j = 5 to pass the test
i = 1
j = 5

hazard_ratio_test(hazard_ratio, i, j, one_hot_train, cph)

#### Expected Output:
```
15.029017732492221 

 All tests passed.
```

**Question:** 

Is `case_1` or `case_2` at greater risk? 

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Check your answer!</b></font>
</summary>
<p>
<ul>
<ul>
    Important! The following answer only applies if you picked i = 1 and j = 5
    <li>You should see that `case_1` is at higher risk.</li>
    <li>The hazard ratio of case 1 / case 2 is greater than 1, so case 1 had a higher hazard relative to case 2</li>
</ul>
</p>

Inspect different pairs, and see if you can figure out which patient is more at risk.

In [ ]:
i = 4
case_1 = one_hot_train.iloc[i, :].drop(['time', 'status'])

j = 7
case_2 = one_hot_train.iloc[j, :].drop(['time', 'status'])

print("Case 1\n\n", case_1, "\n")
print("Case 2\n\n", case_2, "\n")
print("Hazard Ratio:", hazard_ratio(case_1.values, case_2.values, cph.params_.values))

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Check your answer!</b></font>
</summary>
<p>
<ul>
<ul>
    Important! The following answer only applies if you picked i = 4 and j = 7
    <li>You should see that `case_2` is at higher risk.</li>
    <li>The hazard ratio of case 1 / case 2 is less than 1, so case 2 had a higher hazard relative to case 1</li>
</ul>
</p>

<a name='7'></a>
## 7. Harrell's C-index

To evaluate how good our model is performing, we will write our own version of the C-index. Similar to the week 1 case, C-index in the survival context is the probability that, given a randomly selected pair of individuals, the one who died sooner has a higher risk score. 

However, we need to take into account censoring. Imagine a pair of patients, $A$ and $B$. 

#### Scenario 1
- A was censored at time $t_A$ 
- B died at $t_B$
- $t_A < t_B$. 

Because of censoring, we can't say whether $A$ or $B$ should have a higher risk score. 

#### Scenario 2
Now imagine that $t_A > t_B$.

- A was censored at time $t_A$ 
- B died at $t_B$
- $t_A > t_B$

Now we can definitively say that $B$ should have a higher risk score than $A$, since we know for a fact that $A$ lived longer. 

Therefore, when we compute our C-index
- We should only consider pairs where at most one person is censored
- If they are censored, then their censored time should occur *after* the other person's time of death. 

The metric we get if we use this rule is called **Harrel's C-index**.

Note that in this case, being censored at time $t$ means that the true death time was some time AFTER time $t$ and not at $t$. 
- Therefore if $t_A = t_B$ and A was censored:
    - Then $A$ actually lived longer than $B$. 
    - This will effect how you deal with ties in the exercise below!



<a name='ex-3'></a>
### Exercise 3 - harrell_c
Fill in the function below to compute Harrel's C-index.

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<p>
<ul>
    <li>If you get a division by zero error, consider checking how you count when a pair is permissible (in the case where one patient is censored and the other is not censored).</li>
</ul>
</p>

In [ ]:

def harrell_c(y_true, scores, event):
    '''
    Compute Harrel C-index given true event/censoring times,
    model output, and event indicators.
    
    Args:
        y_true (array): array of true event times
        scores (array): model risk scores
        event (array): indicator, 1 if event occurred at that index, 0 for censorship
    Returns:
        result (float): C-index metric
    '''
    
    n = len(y_true)
    assert (len(scores) == n and len(event) == n)
    
    concordant = 0.0
    permissible = 0.0
    ties = 0.0
    
    result = 0.0
    
    # use double for loop to go through cases
    for i in range(n):
        # set lower bound on j to avoid double counting
        for j in range(i+1, n):
            
            # check if at most one is censored
            if event[i] == 1 or event[j] == 1:
            
                # check if neither are censored
                if event[i] == 1 and event[j] == 1:
                    
                    permissible += 1.0
                    
                    # check if scores are tied
                    if scores[i] == scores[j]:
                        ties += 1.0
                    
                    # check for concordant
                    elif y_true[i] < y_true[j] and scores[i] > scores[j]:
                        concordant += 1.0
                    elif y_true[i] > y_true[j] and scores[i] < scores[j]:
                        concordant += 1.0
                
                # check if one is censored
                elif event[i] != event[j]:
                    
                    # get censored index
                    censored = j
                    uncensored = i
                    
                    if event[i] == 0:
                        censored = i
                        uncensored = j
                        
                    # check if permissible
                    # Note: in this case, we are assuming that censored at a time
                    # means that you did NOT die at that time. That is, if you
                    # live until time 30 and have event = 0, then you lived THROUGH
                    # time 30.
                    if y_true[uncensored] <= y_true[censored]:
                        permissible += 1.0
                        
                        # check if scores are tied
                        if scores[uncensored] == scores[censored]:
                            # update ties 
                            ties += 1.0
                            
                        # check if scores are concordant 
                        if scores[uncensored] > scores[censored]:
                            concordant += 1.0
    
    # set result to c-index computed from number of concordant pairs,
    # number of ties, and number of permissible pairs (REPLACE 0 with your code)  
    result = (concordant + 0.5*ties) / permissible
    
    return result   

In [ ]:
# test cell ex1 - do not modify this test cell
harrell_c_test(harrell_c)    

#### Expected Output:
```
Test Case 1
Output:  1.0

Test Case 2
Output:  0.0

Test Case 3
Output:  1.0

Test Case 4
Output:  0.75

Test Case 5
Output:  0.5833333333333334

Tes Case 6
Output:  1.0 

 All tests passed.
```

Now use the Harrell's C-index function to evaluate the cox model on our data sets.

In [ ]:
# Train
scores = cph.predict_partial_hazard(one_hot_train)
cox_train_scores = harrell_c(one_hot_train['time'].values, scores.values, one_hot_train['status'].values)
# Validation
scores = cph.predict_partial_hazard(one_hot_val)
cox_val_scores = harrell_c(one_hot_val['time'].values, scores.values, one_hot_val['status'].values)
# Test
scores = cph.predict_partial_hazard(one_hot_test)
cox_test_scores = harrell_c(one_hot_test['time'].values, scores.values, one_hot_test['status'].values)

print("Train:", cox_train_scores)
print("Val:", cox_val_scores)
print("Test:", cox_test_scores)

What do these values tell us ?

<a name='8'></a>
## 8. Random Survival Forests

This performed well, but you have a hunch you can squeeze out better performance by using a machine learning approach. You decide to use a Random Survival Forest. To do this, you can use the `RandomForestSRC` package in R. To call R function from Python, we'll use the `r2py` package. Run the following cell to import the necessary requirements. 


In [ ]:
#!pip install rpy2

In [ ]:
%load_ext rpy2.ipython
%R require(ggplot2)

# from rpy2.robjects.packages import importr
# # import R's "base" package
# base = importr('base')

# # import R's "utils" package
# utils = importr('utils')

# # import rpy2's package module
# import rpy2.robjects.packages as rpackages

# ## forest = rpackages.importr('randomForestSRC', lib_loc='R')
# forest = rpackages.importr('randomForestSRC') 

# from rpy2 import robjects as ro
# R = ro.r

# from rpy2.robjects import pandas2ri
# pandas2ri.activate()

#-------------------------------------------

# import rpy2's package module
import rpy2.robjects.packages as rpackages

# import R's utility package
utils = rpackages.importr('utils')

# select a mirror for R packages
utils.chooseCRANmirror(ind=1) # select the first mirror in the list

# R package names
packnames = ('ggplot2', 'randomForestSRC')

# R vector of strings
from rpy2.robjects.vectors import StrVector

# Selectively install what needs to be install.
# We are fancy, just because we can.
names_to_install = [x for x in packnames if not rpackages.isinstalled(x)]
if len(names_to_install) > 0:
    utils.install_packages(StrVector(names_to_install))



forest = rpackages.importr('randomForestSRC') 

from rpy2 import robjects as ro
R = ro.r

from rpy2.robjects import pandas2ri
pandas2ri.activate()

Instead of encoding our categories as binary features, we can use the original dataframe since trees deal well with raw categorical data (can you think why this might be?).

Run the code cell below to build your forest.

In [ ]:
model = forest.rfsrc(ro.Formula('Surv(time, status) ~ .'), data=df_train, ntree=300, nodedepth=5, seed=-1)

In [ ]:
print(model)

Finally, let's evaluate on our validation and test sets, and compare it with our Cox model.

In [ ]:
result = R.predict(model, newdata=df_val)
scores = np.array(result.rx('predicted')[0])

print("Cox Model Validation Score:", cox_val_scores)
print("Survival Forest Validation Score:", harrell_c(df_val['time'].values, scores, df_val['status'].values))

Cox Model Validation Score: 0.8544776119402985
Survival Forest Validation Score: 0.8296019900497512


In [ ]:
result = R.predict(model, newdata=df_test)
scores = np.array(result.rx('predicted')[0])

print("Cox Model Test Score:", cox_test_scores)
print("Survival Forest Validation Score:", harrell_c(df_test['time'].values, scores, df_test['status'].values))

Cox Model Test Score: 0.8478543563068921
Survival Forest Validation Score: 0.8621586475942783


Your random forest model should be outperforming the Cox model slightly. Let's dig deeper to see how they differ.

<a name='9'></a>
## 9. Permutation Method for Interpretation

We'll dig a bit deeper into interpretation methods for forests a bit later, but for now just know that random surival forests come with their own built in variable importance feature. The method is referred to as VIMP, and for the purpose of this section you should just know that higher absolute value of the VIMP means that the variable generally has a larger effect on the model outcome.

Run the next cell to compute and plot VIMP for the random survival forest.

In [ ]:
vimps = np.array(forest.vimp(model).rx('importance')[0])

y = np.arange(len(vimps))
plt.barh(y, np.abs(vimps))
plt.yticks(y, df_train.drop(['time', 'status'], axis=1).columns)
plt.title("VIMP (absolute value)")
plt.show()